In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
import json

# Wastewater viral load data

In [2]:
wastewater_data = pd.read_csv('data/COVID-19_rioolwaterdata_gemeentenweek.csv', delimiter=';')

In [3]:
wastewater_data = wastewater_data.drop(columns=['Version', 'Date_of_report'])

In [4]:
wastewater_data = wastewater_data[wastewater_data['Year'] <= 2022]
wastewater_data = wastewater_data.sort_values(by=['Start_date', 'Region_code'])
# rename column
wastewater_data = wastewater_data.rename(columns={'Region_code': 'Municipality_code', 'Region_name': 'Municipality_name'})
# query first 5 rows of data to see the structure
wastewater_data.head()

,Year,Week,Start_date,End_date,Municipality_code,Municipality_name,RNA_flow_per_100000_weeklymean
112,2020,37,2020-09-07,2020-09-13,GM0014,Groningen,2.144168e+12
10,2020,37,2020-09-07,2020-09-13,GM0034,Almere,1.747768e+13
268,2020,37,2020-09-07,2020-09-13,GM0037,Stadskanaal,0.000000e+00
296,2020,37,2020-09-07,2020-09-13,GM0047,Veendam,2.162135e+12
341,2020,37,2020-09-07,2020-09-13,GM0050,Zeewolde,8.303747e+09


In [5]:
# select row that has NA value in 'RNA_flow_per_100000_weeklymean' column
wastewater_data[wastewater_data['RNA_flow_per_100000_weeklymean'].isna()]

,Year,Week,Start_date,End_date,Municipality_code,Municipality_name,RNA_flow_per_100000_weeklymean
114,2020,37,2020-09-07,2020-09-13,GM0158,Haaksbergen,NaN
83,2020,37,2020-09-07,2020-09-13,GM0225,Druten,NaN
84,2020,37,2020-09-07,2020-09-13,GM0226,Duiven,NaN
93,2020,37,2020-09-07,2020-09-13,GM0230,Elburg,NaN
251,2020,37,2020-09-07,2020-09-13,GM0277,Rozendaal,NaN
...,...,...,...,...,...,...,...
41539,2022,51,2022-12-19,2022-12-25,GM0376,Blaricum,NaN
41660,2022,51,2022-12-19,2022-12-25,GM0417,Laren,NaN
41729,2022,51,2022-12-19,2022-12-25,GM0439,Purmerend,NaN
41755,2022,51,2022-12-19,2022-12-25,GM0610,Sliedrecht,NaN


# Hospitalization Data

In [6]:
hospitalization_data1 = pd.read_csv('data/COVID-19_ziekenhuisopnames_tm_03102021.csv', delimiter=';')
hospitalization_data2 = pd.read_csv('data/COVID-19_ziekenhuisopnames.csv', delimiter=';')

In [7]:
# since hospitalization data has 2 files, we concat two dataframes together
hospitalization_data = pd.concat([hospitalization_data1, hospitalization_data2], axis=0)
hospitalization_data.head()

,Version,Date_of_report,Date_of_statistics,Municipality_code,Municipality_name,Security_region_code,Security_region_name,Hospital_admission_notification,Hospital_admission
0,7,2022-09-02T00:30:00Z,2020-02-27,GM0014,Groningen,VR01,Groningen,0,0
1,7,2022-09-02T00:30:00Z,2020-02-27,GM0034,Almere,VR25,Flevoland,0,0
2,7,2022-09-02T00:30:00Z,2020-02-27,GM0037,Stadskanaal,VR01,Groningen,0,0
3,7,2022-09-02T00:30:00Z,2020-02-27,GM0047,Veendam,VR01,Groningen,0,0
4,7,2022-09-02T00:30:00Z,2020-02-27,GM0050,Zeewolde,VR25,Flevoland,0,0


In [8]:
# select row that has NA value in 'Municipality_code' column
hospitalization_data[hospitalization_data['Municipality_code'].isna()]

,Version,Date_of_report,Date_of_statistics,Municipality_code,Municipality_name,Security_region_code,Security_region_name,Hospital_admission_notification,Hospital_admission
342,7,2022-09-02T00:30:00Z,2020-02-27,NaN,NaN,NaN,NaN,0,0
685,7,2022-09-02T00:30:00Z,2020-02-28,NaN,NaN,NaN,NaN,0,0
1028,7,2022-09-02T00:30:00Z,2020-02-29,NaN,NaN,NaN,NaN,0,0
1371,7,2022-09-02T00:30:00Z,2020-03-01,NaN,NaN,NaN,NaN,0,0
1714,7,2022-09-02T00:30:00Z,2020-03-02,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...
302525,7,2024-03-13 00:30:00,2024-03-06,NaN,NaN,NaN,NaN,0,0
302868,7,2024-03-13 00:30:00,2024-03-07,NaN,NaN,NaN,NaN,0,0
303211,7,2024-03-13 00:30:00,2024-03-09,NaN,NaN,NaN,NaN,0,0
303554,7,2024-03-13 00:30:00,2024-03-11,NaN,NaN,NaN,NaN,0,4


In [9]:
hospitalization_data = hospitalization_data.drop(columns=['Version', 'Date_of_report'])
# create new column to store 'year' from 'Date_of_statistics' column
hospitalization_data['Year'] = pd.to_datetime(hospitalization_data['Date_of_statistics']).dt.year
# select data from 2020-2022
hospitalization_data = hospitalization_data[hospitalization_data['Year'] <= 2022]
# create new column to store 'week' (week number) from 'Date_of_statistics' column
hospitalization_data['Week'] = pd.to_datetime(hospitalization_data['Date_of_statistics']).dt.isocalendar().week
# hospitalization_data = hospitalization_data[~(((hospitalization_data['Year']==2020) & (hospitalization_data['Week']<37)) | ((hospitalization_data['Year']==2022)&(hospitalization_data['Week']>26)))]
hospitalization_data.head()

,Date_of_statistics,Municipality_code,Municipality_name,Security_region_code,Security_region_name,Hospital_admission_notification,Hospital_admission,Year,Week
0,2020-02-27,GM0014,Groningen,VR01,Groningen,0,0,2020,9
1,2020-02-27,GM0034,Almere,VR25,Flevoland,0,0,2020,9
2,2020-02-27,GM0037,Stadskanaal,VR01,Groningen,0,0,2020,9
3,2020-02-27,GM0047,Veendam,VR01,Groningen,0,0,2020,9
4,2020-02-27,GM0050,Zeewolde,VR25,Flevoland,0,0,2020,9


In [10]:
# create new column to store 'date' from 'Date_of_statistics' column
hospitalization_data['date_num'] = pd.to_datetime(hospitalization_data['Date_of_statistics']).dt.day
# manually reassign the year to some row of data (e.g. 2021-01-01 is still in week 53 of 2020)
hospitalization_data.loc[(hospitalization_data['Year']==2021)&(hospitalization_data['Week']==53), 'Year'] = 2020
hospitalization_data.loc[(hospitalization_data['Year']==2022)&(hospitalization_data['Week']==52)&(hospitalization_data['date_num']<3), 'Year'] = 2021

In [11]:
hospitalization_data[hospitalization_data['date_num']<3]

,Date_of_statistics,Municipality_code,Municipality_name,Security_region_code,Security_region_name,Hospital_admission_notification,Hospital_admission,Year,Week,date_num
1029,2020-03-01,GM0014,Groningen,VR01,Groningen,0,0,2020,9,1
1030,2020-03-01,GM0034,Almere,VR25,Flevoland,0,0,2020,9,1
1031,2020-03-01,GM0037,Stadskanaal,VR01,Groningen,0,0,2020,9,1
1032,2020-03-01,GM0047,Veendam,VR01,Groningen,0,0,2020,9,1
1033,2020-03-01,GM0050,Zeewolde,VR25,Flevoland,0,0,2020,9,1
...,...,...,...,...,...,...,...,...,...,...
145770,2022-12-02,GM1980,Dijk en Waard,VR10,Noord-Holland-Noord,0,0,2022,48,2
145771,2022-12-02,GM1982,Land van Cuijk,VR21,Brabant-Noord,0,0,2022,48,2
145772,2022-12-02,GM1991,Maashorst,VR21,Brabant-Noord,0,1,2022,48,2
145773,2022-12-02,GM1992,Voorne aan Zee,VR17,Rotterdam-Rijnmond,0,1,2022,48,2


In [12]:
hospitalization_data[(hospitalization_data['Year']==2020)&(hospitalization_data['Week']==53)]

,Date_of_statistics,Municipality_code,Municipality_name,Security_region_code,Security_region_name,Hospital_admission_notification,Hospital_admission,Year,Week,date_num
104615,2020-12-28,GM0014,Groningen,VR01,Groningen,4,2,2020,53,28
104616,2020-12-28,GM0034,Almere,VR25,Flevoland,2,5,2020,53,28
104617,2020-12-28,GM0037,Stadskanaal,VR01,Groningen,0,0,2020,53,28
104618,2020-12-28,GM0047,Veendam,VR01,Groningen,2,2,2020,53,28
104619,2020-12-28,GM0050,Zeewolde,VR25,Flevoland,0,0,2020,53,28
...,...,...,...,...,...,...,...,...,...,...
107011,2021-01-03,GM1980,Dijk en Waard,VR10,Noord-Holland-Noord,1,5,2020,53,3
107012,2021-01-03,GM1982,Land van Cuijk,VR21,Brabant-Noord,3,1,2020,53,3
107013,2021-01-03,GM1991,Maashorst,VR21,Brabant-Noord,0,0,2020,53,3
107014,2021-01-03,GM1992,Voorne aan Zee,VR17,Rotterdam-Rijnmond,0,1,2020,53,3


In [13]:
# aggregate the data per week and municipality
temp_df1 = hospitalization_data.groupby(['Municipality_code', 'Year', 'Week'])['Hospital_admission_notification'].sum().reset_index()
temp_df2 = hospitalization_data.groupby(['Municipality_code', 'Year', 'Week'])['Hospital_admission'].sum().reset_index()

In [14]:
# join dataframe by using 'Municipality_code', 'Year', and 'Week' column
final_hospital_data = pd.merge(temp_df1, temp_df2, how="inner", on=['Municipality_code', 'Year', 'Week'])
final_hospital_data

,Municipality_code,Year,Week,Hospital_admission_notification,Hospital_admission
0,GM0014,2020,9,0,0
1,GM0014,2020,10,0,0
2,GM0014,2020,11,0,0
3,GM0014,2020,12,0,1
4,GM0014,2020,13,9,8
...,...,...,...,...,...
50953,GM1992,2022,48,4,3
50954,GM1992,2022,49,2,2
50955,GM1992,2022,50,2,5
50956,GM1992,2022,51,10,7


In [15]:
# create new dataframe
mun_df = pd.DataFrame()

In [16]:
mun_df = hospitalization_data.drop_duplicates('Municipality_code').dropna()
# dataframe store municipality and safety region information
mun_df = mun_df[['Municipality_code', 'Municipality_name', 'Security_region_code', 'Security_region_name']]

In [17]:
# join dataframe by using 'Municipality_code' (add 'Municipality_name', 'Security_region_code', 'Security_region_name' to 'final_hospital_data' dataframe)
final_hospital_data = pd.merge(final_hospital_data, mun_df, how="inner", on=['Municipality_code'])
final_hospital_data

,Municipality_code,Year,Week,Hospital_admission_notification,Hospital_admission,Municipality_name,Security_region_code,Security_region_name
0,GM0014,2020,9,0,0,Groningen,VR01,Groningen
1,GM0014,2020,10,0,0,Groningen,VR01,Groningen
2,GM0014,2020,11,0,0,Groningen,VR01,Groningen
3,GM0014,2020,12,0,1,Groningen,VR01,Groningen
4,GM0014,2020,13,9,8,Groningen,VR01,Groningen
...,...,...,...,...,...,...,...,...
50953,GM1992,2022,48,4,3,Voorne aan Zee,VR17,Rotterdam-Rijnmond
50954,GM1992,2022,49,2,2,Voorne aan Zee,VR17,Rotterdam-Rijnmond
50955,GM1992,2022,50,2,5,Voorne aan Zee,VR17,Rotterdam-Rijnmond
50956,GM1992,2022,51,10,7,Voorne aan Zee,VR17,Rotterdam-Rijnmond


In [22]:
# using 'Year' and 'Week' column to create pandas series that create YearWeek string (e.g. 202009 <- Year=2020 and Week=9)
dates = final_hospital_data['Year']*100+final_hospital_data['Week']
# create 'Start_date' of each week
final_hospital_data['Start_date'] = pd.to_datetime(dates.astype(str) + '1', format='%G%V%u')
# create 'End_date' of each week
final_hospital_data['End_date'] = final_hospital_data['Start_date'] + pd.Timedelta('6d')
final_hospital_data = final_hospital_data.sort_values(by=['Start_date', 'Municipality_code']).reset_index().drop(columns=['index'])

In [23]:
final_hospital_data

,Municipality_code,Year,Week,Hospital_admission_notification,Hospital_admission,Municipality_name,Security_region_code,Security_region_name,Start_date,End_date
0,GM0014,2020,9,0,0,Groningen,VR01,Groningen,2020-02-24,2020-03-01
1,GM0034,2020,9,0,0,Almere,VR25,Flevoland,2020-02-24,2020-03-01
2,GM0037,2020,9,0,0,Stadskanaal,VR01,Groningen,2020-02-24,2020-03-01
3,GM0047,2020,9,0,0,Veendam,VR01,Groningen,2020-02-24,2020-03-01
4,GM0050,2020,9,0,0,Zeewolde,VR25,Flevoland,2020-02-24,2020-03-01
...,...,...,...,...,...,...,...,...,...,...
50953,GM1979,2022,52,0,0,Eemsdelta,VR01,Groningen,2022-12-26,2023-01-01
50954,GM1980,2022,52,5,6,Dijk en Waard,VR10,Noord-Holland-Noord,2022-12-26,2023-01-01
50955,GM1982,2022,52,9,6,Land van Cuijk,VR21,Brabant-Noord,2022-12-26,2023-01-01
50956,GM1991,2022,52,3,4,Maashorst,VR21,Brabant-Noord,2022-12-26,2023-01-01


In [24]:
final_hospital_data[final_hospital_data['Municipality_name'].isna()]

,Municipality_code,Year,Week,Hospital_admission_notification,Hospital_admission,Municipality_name,Security_region_code,Security_region_name,Start_date,End_date


# Reported cases

In [27]:
reported_case_data1 = pd.read_csv('data/COVID-19_aantallen_gemeente_per_dag_tm_03102021.csv', delimiter=';')
reported_case_data2 = pd.read_csv('data/COVID-19_aantallen_gemeente_per_dag.csv', delimiter=';')

In [28]:
# since reported cases data has 2 files, we concat two dataframes together
reported_case_data = pd.concat([reported_case_data1, reported_case_data2], axis=0)
reported_case_data

,Version,Date_of_report,Date_of_publication,Municipality_code,Municipality_name,Province,Security_region_code,Security_region_name,Municipal_health_service,ROAZ_region,Total_reported,Deceased
0,6,2022-09-02 10:00:00,2020-02-28,GM0014,Groningen,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,0,0
1,6,2022-09-02 10:00:00,2020-02-28,GM0034,Almere,Flevoland,VR25,Flevoland,GGD Flevoland,SpoedZorgNet,0,0
2,6,2022-09-02 10:00:00,2020-02-28,GM0037,Stadskanaal,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,0,0
3,6,2022-09-02 10:00:00,2020-02-28,GM0047,Veendam,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,0,0
4,6,2022-09-02 10:00:00,2020-02-28,GM0050,Zeewolde,Flevoland,VR25,Flevoland,GGD Flevoland,SpoedZorgNet,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
201275,7,2023-03-31 10:00:00,2023-03-31,NaN,NaN,Limburg,VR24,Limburg-Zuid,GGD Zuid-Limburg,Netwerk Acute Zorg Limburg,0,9999
201276,7,2023-03-31 10:00:00,2023-03-31,NaN,NaN,Zuid-Holland,VR15,Haaglanden,GGD Haaglanden,Netwerk Acute Zorg West,0,9999
201277,7,2023-03-31 10:00:00,2023-03-31,NaN,NaN,Fryslân,VR02,Fryslân,GGD Fryslân,Acute Zorgnetwerk Noord Nederland,0,9999
201278,7,2023-03-31 10:00:00,2023-03-31,NaN,NaN,Noord-Holland,VR11,Zaanstreek-Waterland,GGD Zaanstreek/Waterland,NaN,0,9999


In [29]:
# this step is same as I did in hospitalization data
reported_case_data = reported_case_data.drop(columns=['Version', 'Date_of_report'])
reported_case_data['Year'] = pd.to_datetime(reported_case_data['Date_of_publication']).dt.year
reported_case_data = reported_case_data[reported_case_data['Year'] <= 2022]
reported_case_data['Week'] = pd.to_datetime(reported_case_data['Date_of_publication']).dt.isocalendar().week
# reported_case_data = reported_case_data[~(((reported_case_data['Year']==2020) & (reported_case_data['Week']<37)) | ((reported_case_data['Year']==2022)&(reported_case_data['Week']>26)))]
reported_case_data

,Date_of_publication,Municipality_code,Municipality_name,Province,Security_region_code,Security_region_name,Municipal_health_service,ROAZ_region,Total_reported,Deceased,Year,Week
0,2020-02-28,GM0014,Groningen,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,0,0,2020,9
1,2020-02-28,GM0034,Almere,Flevoland,VR25,Flevoland,GGD Flevoland,SpoedZorgNet,0,0,2020,9
2,2020-02-28,GM0037,Stadskanaal,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,0,0,2020,9
3,2020-02-28,GM0047,Veendam,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,0,0,2020,9
4,2020-02-28,GM0050,Zeewolde,Flevoland,VR25,Flevoland,GGD Flevoland,SpoedZorgNet,0,0,2020,9
...,...,...,...,...,...,...,...,...,...,...,...,...
167975,2022-12-31,NaN,NaN,Limburg,VR24,Limburg-Zuid,GGD Zuid-Limburg,Netwerk Acute Zorg Limburg,0,0,2022,52
167976,2022-12-31,NaN,NaN,Zuid-Holland,VR15,Haaglanden,GGD Haaglanden,Netwerk Acute Zorg West,1,0,2022,52
167977,2022-12-31,NaN,NaN,Fryslân,VR02,Fryslân,GGD Fryslân,Acute Zorgnetwerk Noord Nederland,0,0,2022,52
167978,2022-12-31,NaN,NaN,Noord-Holland,VR11,Zaanstreek-Waterland,GGD Zaanstreek/Waterland,NaN,0,0,2022,52


In [30]:
# this step is same as I did in hospitalization data
reported_case_data['date_num'] = pd.to_datetime(reported_case_data['Date_of_publication']).dt.day
reported_case_data.loc[(reported_case_data['Year']==2021)&(reported_case_data['Week']==53), 'Year'] = 2020
reported_case_data.loc[(reported_case_data['Year']==2022)&(reported_case_data['Week']==52)&(reported_case_data['date_num']<3), 'Year'] = 2021

In [31]:
reported_case_data[reported_case_data['Security_region_name'].isna()]

,Date_of_publication,Municipality_code,Municipality_name,Province,Security_region_code,Security_region_name,Municipal_health_service,ROAZ_region,Total_reported,Deceased,Year,Week,date_num
364,2020-02-28,NaN,NaN,Noord-Brabant,NaN,NaN,GGD Hart voor Brabant,Netwerk Acute Zorg Brabant,0,0,2020,9,28
371,2020-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2020,9,28
736,2020-02-29,NaN,NaN,Noord-Brabant,NaN,NaN,GGD Hart voor Brabant,Netwerk Acute Zorg Brabant,0,0,2020,9,29
743,2020-02-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2020,9,29
1108,2020-03-01,NaN,NaN,Noord-Brabant,NaN,NaN,GGD Hart voor Brabant,Netwerk Acute Zorg Brabant,0,0,2020,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
167239,2022-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2022,52,29
167602,2022-12-30,NaN,NaN,Noord-Brabant,NaN,NaN,GGD Hart voor Brabant,Netwerk Acute Zorg Brabant,1,0,2022,52,30
167609,2022-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,2022,52,30
167972,2022-12-31,NaN,NaN,Noord-Brabant,NaN,NaN,GGD Hart voor Brabant,Netwerk Acute Zorg Brabant,2,0,2022,52,31


In [32]:
reported_case_data[reported_case_data['Security_region_name'].isna()]['Province'].unique()

array(['Noord-Brabant', nan], dtype=object)

In [33]:
reported_case_data[(reported_case_data['Province'].isna())].sum()

Date_of_publication         2020-02-282020-02-292020-03-012020-03-022020-0...
Municipality_code                                                           0
Municipality_name                                                           0
Province                                                                    0
Security_region_code                                                        0
Security_region_name                                                        0
Municipal_health_service                                                    0
ROAZ_region                                                                 0
Total_reported                                                            133
Deceased                                                                    0
Year                                                                  2097850
Week                                                                    28969
date_num                                                        

In [34]:
# select data that 'Municipality_code' isn't NA
mun_reported_case_data = reported_case_data[~(reported_case_data['Municipality_code'].isna())]

In [35]:
mun_reported_case_data

,Date_of_publication,Municipality_code,Municipality_name,Province,Security_region_code,Security_region_name,Municipal_health_service,ROAZ_region,Total_reported,Deceased,Year,Week,date_num
0,2020-02-28,GM0014,Groningen,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,0,0,2020,9,28
1,2020-02-28,GM0034,Almere,Flevoland,VR25,Flevoland,GGD Flevoland,SpoedZorgNet,0,0,2020,9,28
2,2020-02-28,GM0037,Stadskanaal,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,0,0,2020,9,28
3,2020-02-28,GM0047,Veendam,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,0,0,2020,9,28
4,2020-02-28,GM0050,Zeewolde,Flevoland,VR25,Flevoland,GGD Flevoland,SpoedZorgNet,0,0,2020,9,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
167949,2022-12-31,GM1979,Eemsdelta,Groningen,VR01,Groningen,GGD Groningen,Acute Zorgnetwerk Noord Nederland,2,0,2022,52,31
167950,2022-12-31,GM1980,Dijk en Waard,Noord-Holland,VR10,Noord-Holland-Noord,GGD Hollands-Noorden,Netwerk Acute Zorg Noordwest,8,0,2022,52,31
167951,2022-12-31,GM1982,Land van Cuijk,Noord-Brabant,VR21,Brabant-Noord,GGD Hart voor Brabant,Netwerk Acute Zorg Brabant,5,0,2022,52,31
167952,2022-12-31,GM1991,Maashorst,Noord-Brabant,VR21,Brabant-Noord,GGD Hart voor Brabant,Netwerk Acute Zorg Brabant,1,0,2022,52,31


In [36]:
# aggregate total reported cases and mortality cases per week and municipality
temp_df1 = mun_reported_case_data.groupby(['Municipality_code', 'Year', 'Week'])['Total_reported'].sum().reset_index()
temp_df2 = mun_reported_case_data.groupby(['Municipality_code', 'Year', 'Week'])['Deceased'].sum().reset_index()

In [37]:
# join two dataframe on 'Municipality_code', 'Year', 'Week' column
final_mun_reported_data = pd.merge(temp_df1, temp_df2, how="inner", on=['Municipality_code', 'Year', 'Week'])
final_mun_reported_data

,Municipality_code,Year,Week,Total_reported,Deceased
0,GM0014,2020,9,0,0
1,GM0014,2020,10,0,0
2,GM0014,2020,11,7,0
3,GM0014,2020,12,41,0
4,GM0014,2020,13,39,0
...,...,...,...,...,...
51121,GM1992,2022,48,14,0
51122,GM1992,2022,49,25,0
51123,GM1992,2022,50,20,0
51124,GM1992,2022,51,23,0


In [38]:
# create dataframe that contain unique municipality code (exclude data that has NA value on 'Municipality_code' value)
mun_df2 = mun_reported_case_data.drop_duplicates('Municipality_code').dropna()
mun_df2 = mun_df2[['Municipality_code', 'Municipality_name', 'Security_region_code', 'Security_region_name']]

In [39]:
mun_df2

,Municipality_code,Municipality_name,Security_region_code,Security_region_name
0,GM0014,Groningen,VR01,Groningen
1,GM0034,Almere,VR25,Flevoland
2,GM0037,Stadskanaal,VR01,Groningen
3,GM0047,Veendam,VR01,Groningen
4,GM0050,Zeewolde,VR25,Flevoland
...,...,...,...,...
342,GM1979,Eemsdelta,VR01,Groningen
343,GM1980,Dijk en Waard,VR10,Noord-Holland-Noord
344,GM1982,Land van Cuijk,VR21,Brabant-Noord
345,GM1991,Maashorst,VR21,Brabant-Noord


In [40]:
# join two dataframes to add more municipality information
final_mun_reported_data = pd.merge(final_mun_reported_data, mun_df2, how="inner", on=['Municipality_code'])
final_mun_reported_data

,Municipality_code,Year,Week,Total_reported,Deceased,Municipality_name,Security_region_code,Security_region_name
0,GM0014,2020,9,0,0,Groningen,VR01,Groningen
1,GM0014,2020,10,0,0,Groningen,VR01,Groningen
2,GM0014,2020,11,7,0,Groningen,VR01,Groningen
3,GM0014,2020,12,41,0,Groningen,VR01,Groningen
4,GM0014,2020,13,39,0,Groningen,VR01,Groningen
...,...,...,...,...,...,...,...,...
51121,GM1992,2022,48,14,0,Voorne aan Zee,VR17,Rotterdam-Rijnmond
51122,GM1992,2022,49,25,0,Voorne aan Zee,VR17,Rotterdam-Rijnmond
51123,GM1992,2022,50,20,0,Voorne aan Zee,VR17,Rotterdam-Rijnmond
51124,GM1992,2022,51,23,0,Voorne aan Zee,VR17,Rotterdam-Rijnmond


# Preparation to merge data
- waste water viral load
- hospitalization data
- reported case
- death case

## Date df

In [41]:
# create dataframe that contain 'Start_date' column from '2020-10-05' till '2022-07-03' (weekly)
date_df = pd.DataFrame({'Start_date': pd.date_range(start='2020-10-05', end='2022-07-03', freq='W-MON')})

In [42]:
# create three new columns
date_df['End_date'] = date_df['Start_date'] + pd.Timedelta('6d')
date_df['Year'] = pd.to_datetime(date_df['Start_date']).dt.year
date_df['Week'] = pd.to_datetime(date_df['Start_date']).dt.isocalendar().week

In [43]:
date_df

,Start_date,End_date,Year,Week
0,2020-10-05,2020-10-11,2020,41
1,2020-10-12,2020-10-18,2020,42
2,2020-10-19,2020-10-25,2020,43
3,2020-10-26,2020-11-01,2020,44
4,2020-11-02,2020-11-08,2020,45
...,...,...,...,...
86,2022-05-30,2022-06-05,2022,22
87,2022-06-06,2022-06-12,2022,23
88,2022-06-13,2022-06-19,2022,24
89,2022-06-20,2022-06-26,2022,25


## Municipalities in NL dataframe

In [44]:
# read data from shape file
all_mun_df = gpd.read_file("data/Grenzen_van_alle_Nederlandse_gemeenten_en_provincies/Grenzen_van_alle_Nederlandse_gemeenten_en_provinciesPolygon.shp")

In [45]:
# concat 'GM' to the municipality code (e.g. GM1234)
for i in range(len(all_mun_df)):
    all_mun_df.loc[i, "CODE"] = "GM"+all_mun_df.loc[i, "CODE"]

In [46]:
all_mun_df

,POLY_AREA,PROVCODE,PROVINCIEN,GEMEENTENA,CODE,GML_ID,OBJECTID,GMCODE,tr-2021-52,rna2021-51,tr-2020-46,rna2020-45,tr-2022-19,rna2022-18,geometry
0,1.295331e+08,5.0,Gelderland,Nunspeet,GM0302,idcf22b5f3-d153-4ea8-978b-4fe9e8dc2786,261.0,GM0302,398.549,3.321846e+10,154.393,3.827245e+10,46.677,2.282095e+10,"POLYGON ((187540.926 484932.159, 187659.222 48..."
1,5.650169e+07,10.0,Noord-Brabant,Valkenswaard,GM0858,id3c9f8495-f973-464c-9a40-e70bcd35d413,262.0,GM0858,484.083,5.847034e+10,185.939,5.611317e+10,19.235,1.548888e+10,"POLYGON ((157801.184 369460.522, 157807.384 36..."
2,9.538842e+07,4.0,Overijssel,Wierden,GM0189,idcdbfaa38-94b8-4c96-aac4-31f630aa5b94,263.0,GM0189,666.776,6.436630e+10,433.609,8.074795e+10,24.544,3.775824e+10,"POLYGON ((234354.726 478344.848, 234333.476 47..."
3,6.183056e+07,4.0,Overijssel,Hengelo (O),GM0164,id895b374b-a0ab-4df5-ae0d-698b7d427b71,264.0,GM0164,496.672,4.137643e+10,268.671,8.298401e+10,41.903,2.462429e+10,"POLYGON ((250725.890 472832.259, 250715.563 47..."
4,4.723327e+07,5.0,Gelderland,Renkum,GM0274,id428a7a17-9adc-454a-b68a-5241aa5b3fd0,265.0,GM0274,372.386,4.353861e+10,130.494,6.731668e+10,50.925,2.681317e+10,"POLYGON ((178933.110 442332.070, 178669.610 44..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,5.042307e+07,11.0,Limburg,Gennep,GM0907,id749ef31f-500f-4b8c-93d5-451765d647a4,215.0,GM0907,413.687,5.267961e+10,147.745,5.715011e+10,5.910,2.522403e+10,"POLYGON ((199491.888 410682.389, 199478.873 41..."
348,7.876813e+07,11.0,Limburg,Eijsden-Margraten,GM1903,id547c907f-a6e9-4922-ad5f-5c6ceec856d2,216.0,GM1903,558.833,5.006945e+10,124.185,1.420363e+10,23.285,2.920038e+10,"POLYGON ((180276.453 316142.906, 180272.453 31..."
349,2.540048e+08,9.0,Zeeland,Tholen,GM0716,idec71c934-fd56-4433-add5-752c5d484221,224.0,GM0716,574.601,7.124396e+10,-9999.000,-9.999000e+03,15.530,1.244912e+10,"POLYGON ((64906.104 405680.606, 64907.132 4057..."
350,1.665180e+08,4.0,Overijssel,Dalfsen,GM0148,id20b3855b-df5d-4c8a-82b8-89dfe1d6d5f3,225.0,GM0148,580.684,4.565146e+10,223.878,2.289193e+10,34.981,1.654528e+10,"POLYGON ((219170.413 510005.152, 219171.547 51..."


In [48]:
# rename column
mun_code_df = all_mun_df[['GEMEENTENA', 'CODE']]
mun_code_df = mun_code_df.rename({'GEMEENTENA': 'Municipality_name', 'CODE': 'Municipality_code'}, axis=1)

## Get population number data from CBS

In [49]:
# Get total population of each municipality from API request
api_url = 'https://opendata.cbs.nl/ODataApi/odata/03759ned/TypedDataSet'

for i in range(len(mun_code_df)):
    code = mun_code_df.loc[i, "Municipality_code"]
    try:
        r = requests.get(api_url, params={
            '$filter': "RegioS eq '" + code + "' and BurgerlijkeStaat eq 'T001019' and Geslacht eq 'T001038' and Leeftijd eq '10000'",
            '$select': "Perioden,BevolkingOp1Januari_1"
        })
        r.raise_for_status()
        data_json = json.loads(r.text)
        print(i)
        pop_arr = data_json['value']
        for j in range(len(pop_arr)):
            pop_no = pop_arr[j]['BevolkingOp1Januari_1']
            # this condition handle the case that some total population number isn't available in year 2020
            # so I get the data from year 2021 instead
            if((pop_arr[j]['Perioden'].startswith("2020")) and (pop_no != None)):
                mun_code_df.loc[i, "Population"] = pop_no
                break
            elif((pop_arr[j]['Perioden'].startswith("2021")) and (pop_no != None)):
                mun_code_df.loc[i, "Population"] = pop_no
                break
    except r.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [50]:
# if there is NA value in 'Population' column, NA value is replaced by -9999
mun_code_df['Population'] = mun_code_df['Population'].fillna(-9999).astype(int)

In [52]:
mun_code_df[mun_code_df['Population']==-9999]

,Municipality_name,Municipality_code,Population


In [53]:
mun_code_df

,Municipality_name,Municipality_code,Population
0,Nunspeet,GM0302,27851
1,Valkenswaard,GM0858,31193
2,Wierden,GM0189,24446
3,Hengelo (O),GM0164,81140
4,Renkum,GM0274,31419
...,...,...,...
347,Gennep,GM0907,16921
348,Eijsden-Margraten,GM1903,25768
349,Tholen,GM0716,25757
350,Dalfsen,GM0148,28587


## Population Density from CBS

In [54]:
popdense_df = pd.read_csv("data/Bevolkingsdichtheid2023.csv", delimiter=";")

In [55]:
popdense_df = popdense_df.rename({"Gemeente": "Municipality_name", "Inwoners per km² land": "popdense"}, axis=1)

In [56]:
# format the string
for i in range(len(popdense_df)):
    temp = popdense_df.loc[i, "popdense"]
    if (len(temp) == 5):
        popdense_df.loc[i, "popdense"] = temp.replace(" ","")

In [57]:
popdense_df

,Municipality_name,popdense
0,'s-Gravenhage (gemeente),6827
1,Leiden,5816
2,Haarlem,5662
3,Amsterdam,4880
4,Capelle aan den IJssel,4777
...,...,...
337,Ameland,68
338,Rozendaal,63
339,Terschelling,57
340,Vlieland,31


In [58]:
# convert string to int
popdense_df['popdense'] = popdense_df['popdense'].astype(int)

In [59]:
# manually replace some mismatch municipality name (this will be useful when we need to join the data)
popdense_df['Municipality_name'] = np.where(popdense_df['Municipality_name'] == "Hengelo (O.)", "Hengelo (O)", popdense_df['Municipality_name'])
popdense_df['Municipality_name'] = np.where(popdense_df['Municipality_name'] == "Bergen (NH.)", "Bergen (NH)", popdense_df['Municipality_name'])
popdense_df['Municipality_name'] = np.where(popdense_df['Municipality_name'] == "Bergen (L.)", "Bergen (L)", popdense_df['Municipality_name'])
for i in range(len(popdense_df)):
    temp = popdense_df.loc[i, "Municipality_name"]
    if((temp != "Hengelo (O)") and (temp != "Bergen (NH)") and (temp != "Bergen (L)")):
        if("(" in temp):
            popdense_df.loc[i, "Municipality_name"] = temp.split(" ")[0]

In [60]:
mun_code_df2 = pd.merge(mun_code_df, popdense_df, how='left', on=['Municipality_name'])

In [61]:
mun_code_df2

,Municipality_name,Municipality_code,Population,popdense
0,Nunspeet,GM0302,27851,223.0
1,Valkenswaard,GM0858,31193,574.0
2,Wierden,GM0189,24446,263.0
3,Hengelo (O),GM0164,81140,1353.0
4,Renkum,GM0274,31419,685.0
...,...,...,...,...
347,Gennep,GM0907,16921,373.0
348,Eijsden-Margraten,GM1903,25768,335.0
349,Tholen,GM0716,25757,183.0
350,Dalfsen,GM0148,28587,179.0


## Merge Dataframe

In [63]:
# merge initial date and municipality dataframe by using cross join (getting every possible of pair combination)
# so each week has 352 records (352 municipality)
start_df = pd.merge(date_df, mun_code_df2, how='cross')

In [64]:
start_df

,Start_date,End_date,Year,Week,Municipality_name,Municipality_code,Population,popdense
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,27851,223.0
1,2020-10-05,2020-10-11,2020,41,Valkenswaard,GM0858,31193,574.0
2,2020-10-05,2020-10-11,2020,41,Wierden,GM0189,24446,263.0
3,2020-10-05,2020-10-11,2020,41,Hengelo (O),GM0164,81140,1353.0
4,2020-10-05,2020-10-11,2020,41,Renkum,GM0274,31419,685.0
...,...,...,...,...,...,...,...,...
32027,2022-06-27,2022-07-03,2022,26,Gennep,GM0907,16921,373.0
32028,2022-06-27,2022-07-03,2022,26,Eijsden-Margraten,GM1903,25768,335.0
32029,2022-06-27,2022-07-03,2022,26,Tholen,GM0716,25757,183.0
32030,2022-06-27,2022-07-03,2022,26,Dalfsen,GM0148,28587,179.0


In [65]:
# merge wastewater viral load data to the initial dataframe using left join
merge_ww_df = pd.merge(start_df, wastewater_data, how='left', on=['Municipality_code', 'Year', 'Week'])

In [66]:
# drop some duplicated columns
merge_ww_df = merge_ww_df.drop(columns=['Start_date_y', 'End_date_y', 'Municipality_name_y'])

In [67]:
merge_ww_df

,Start_date_x,End_date_x,Year,Week,Municipality_name_x,Municipality_code,Population,popdense,RNA_flow_per_100000_weeklymean
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,27851,223.0,9.017750e+12
1,2020-10-05,2020-10-11,2020,41,Valkenswaard,GM0858,31193,574.0,NaN
2,2020-10-05,2020-10-11,2020,41,Wierden,GM0189,24446,263.0,2.173360e+13
3,2020-10-05,2020-10-11,2020,41,Hengelo (O),GM0164,81140,1353.0,4.807103e+13
4,2020-10-05,2020-10-11,2020,41,Renkum,GM0274,31419,685.0,2.216738e+13
...,...,...,...,...,...,...,...,...,...
32027,2022-06-27,2022-07-03,2022,26,Gennep,GM0907,16921,373.0,9.226679e+13
32028,2022-06-27,2022-07-03,2022,26,Eijsden-Margraten,GM1903,25768,335.0,9.874191e+13
32029,2022-06-27,2022-07-03,2022,26,Tholen,GM0716,25757,183.0,1.062250e+14
32030,2022-06-27,2022-07-03,2022,26,Dalfsen,GM0148,28587,179.0,1.178476e+14


In [68]:
# merge hospitalization data to the initial dataframe using left join
merge_hp_df = pd.merge(start_df, final_hospital_data, how='left', on=['Municipality_code', 'Year', 'Week'])

In [69]:
# drop some duplicated columns
merge_hp_df = merge_hp_df.drop(columns=['Start_date_y', 'End_date_y', 'Municipality_name_y'])

In [70]:
merge_hp_df

,Start_date_x,End_date_x,Year,Week,Municipality_name_x,Municipality_code,Population,popdense,Hospital_admission_notification,Hospital_admission,Security_region_code,Security_region_name
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,27851,223.0,3.0,0.0,VR06,Noord- en Oost-Gelderland
1,2020-10-05,2020-10-11,2020,41,Valkenswaard,GM0858,31193,574.0,0.0,1.0,VR22,Brabant-Zuidoost
2,2020-10-05,2020-10-11,2020,41,Wierden,GM0189,24446,263.0,2.0,2.0,VR05,Twente
3,2020-10-05,2020-10-11,2020,41,Hengelo (O),GM0164,81140,1353.0,11.0,11.0,VR05,Twente
4,2020-10-05,2020-10-11,2020,41,Renkum,GM0274,31419,685.0,1.0,0.0,VR07,Gelderland-Midden
...,...,...,...,...,...,...,...,...,...,...,...,...
32027,2022-06-27,2022-07-03,2022,26,Gennep,GM0907,16921,373.0,1.0,0.0,VR23,Limburg-Noord
32028,2022-06-27,2022-07-03,2022,26,Eijsden-Margraten,GM1903,25768,335.0,0.0,0.0,VR24,Limburg-Zuid
32029,2022-06-27,2022-07-03,2022,26,Tholen,GM0716,25757,183.0,0.0,0.0,VR19,Zeeland
32030,2022-06-27,2022-07-03,2022,26,Dalfsen,GM0148,28587,179.0,1.0,1.0,VR04,IJsselland


In [71]:
# merge reported cases and mortality cases data to the initial dataframe using left join
merge_case_df = pd.merge(start_df, final_mun_reported_data, how='left', on=['Municipality_code', 'Year', 'Week'])

In [72]:
merge_case_df = merge_case_df.drop(columns=['Municipality_name_y'])

In [73]:
merge_case_df

,Start_date,End_date,Year,Week,Municipality_name_x,Municipality_code,Population,popdense,Total_reported,Deceased,Security_region_code,Security_region_name
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,27851,223.0,40.0,0.0,VR06,Noord- en Oost-Gelderland
1,2020-10-05,2020-10-11,2020,41,Valkenswaard,GM0858,31193,574.0,68.0,0.0,VR22,Brabant-Zuidoost
2,2020-10-05,2020-10-11,2020,41,Wierden,GM0189,24446,263.0,45.0,0.0,VR05,Twente
3,2020-10-05,2020-10-11,2020,41,Hengelo (O),GM0164,81140,1353.0,213.0,2.0,VR05,Twente
4,2020-10-05,2020-10-11,2020,41,Renkum,GM0274,31419,685.0,96.0,2.0,VR07,Gelderland-Midden
...,...,...,...,...,...,...,...,...,...,...,...,...
32027,2022-06-27,2022-07-03,2022,26,Gennep,GM0907,16921,373.0,32.0,0.0,VR23,Limburg-Noord
32028,2022-06-27,2022-07-03,2022,26,Eijsden-Margraten,GM1903,25768,335.0,46.0,0.0,VR24,Limburg-Zuid
32029,2022-06-27,2022-07-03,2022,26,Tholen,GM0716,25757,183.0,27.0,0.0,VR19,Zeeland
32030,2022-06-27,2022-07-03,2022,26,Dalfsen,GM0148,28587,179.0,49.0,1.0,VR04,IJsselland


In [74]:
# merge all dataframes from previous steps together (wastewater viral load, hospitalization case, reported&mortality cases) 
final_df = pd.merge(merge_ww_df, merge_hp_df, how='inner', on=['Municipality_code', 'Year', 'Week'])
final_df = pd.merge(final_df, merge_case_df, how='inner', on=['Municipality_code', 'Year', 'Week'])
final_df = final_df.drop(columns=['Start_date_x_y', 'End_date_x_y', 'Municipality_name_x_y', 'Security_region_code_x', 'Security_region_name_x', 'Start_date', 'End_date', 'Municipality_name_x'])

In [75]:
final_df = final_df.rename({'Start_date_x_x': 'Start_date', 
                            'End_date_x_x': 'End_date', 
                            'Municipality_name_x_x': 'Municipality_name', 
                            'Security_region_code_y': 'Security_region_code',
                            'Security_region_name_y': 'Security_region_name'}, axis=1)

In [76]:
# if the data in specific column is NA, fill -9999 instead
final_df['Hospital_admission_notification'] = final_df['Hospital_admission_notification'].fillna(-9999).astype(int)
final_df['Hospital_admission'] = final_df['Hospital_admission'].fillna(-9999).astype(int)
final_df['Total_reported'] = final_df['Total_reported'].fillna(-9999).astype(int)
final_df['Deceased'] = final_df['Deceased'].fillna(-9999).astype(int)
final_df['RNA_flow_per_100000_weeklymean'] = final_df['RNA_flow_per_100000_weeklymean'].fillna(-9999)

In [77]:
final_df

,Start_date,End_date,Year,Week,Municipality_name,Municipality_code,Population_x,popdense_x,RNA_flow_per_100000_weeklymean,Population_y,popdense_y,Hospital_admission_notification,Hospital_admission,Population,popdense,Total_reported,Deceased,Security_region_code,Security_region_name
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,27851,223.0,9.017750e+12,27851,223.0,3,0,27851,223.0,40,0,VR06,Noord- en Oost-Gelderland
1,2020-10-05,2020-10-11,2020,41,Valkenswaard,GM0858,31193,574.0,-9.999000e+03,31193,574.0,0,1,31193,574.0,68,0,VR22,Brabant-Zuidoost
2,2020-10-05,2020-10-11,2020,41,Wierden,GM0189,24446,263.0,2.173360e+13,24446,263.0,2,2,24446,263.0,45,0,VR05,Twente
3,2020-10-05,2020-10-11,2020,41,Hengelo (O),GM0164,81140,1353.0,4.807103e+13,81140,1353.0,11,11,81140,1353.0,213,2,VR05,Twente
4,2020-10-05,2020-10-11,2020,41,Renkum,GM0274,31419,685.0,2.216738e+13,31419,685.0,1,0,31419,685.0,96,2,VR07,Gelderland-Midden
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32027,2022-06-27,2022-07-03,2022,26,Gennep,GM0907,16921,373.0,9.226679e+13,16921,373.0,1,0,16921,373.0,32,0,VR23,Limburg-Noord
32028,2022-06-27,2022-07-03,2022,26,Eijsden-Margraten,GM1903,25768,335.0,9.874191e+13,25768,335.0,0,0,25768,335.0,46,0,VR24,Limburg-Zuid
32029,2022-06-27,2022-07-03,2022,26,Tholen,GM0716,25757,183.0,1.062250e+14,25757,183.0,0,0,25757,183.0,27,0,VR19,Zeeland
32030,2022-06-27,2022-07-03,2022,26,Dalfsen,GM0148,28587,179.0,1.178476e+14,28587,179.0,1,1,28587,179.0,49,1,VR04,IJsselland


In [78]:
final_df = final_df.drop(columns=['Population_y', 'Population_x', 'popdense_x', 'popdense_y'])

In [79]:
final_df

,Start_date,End_date,Year,Week,Municipality_name,Municipality_code,RNA_flow_per_100000_weeklymean,Hospital_admission_notification,Hospital_admission,Population,popdense,Total_reported,Deceased,Security_region_code,Security_region_name
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,9.017750e+12,3,0,27851,223.0,40,0,VR06,Noord- en Oost-Gelderland
1,2020-10-05,2020-10-11,2020,41,Valkenswaard,GM0858,-9.999000e+03,0,1,31193,574.0,68,0,VR22,Brabant-Zuidoost
2,2020-10-05,2020-10-11,2020,41,Wierden,GM0189,2.173360e+13,2,2,24446,263.0,45,0,VR05,Twente
3,2020-10-05,2020-10-11,2020,41,Hengelo (O),GM0164,4.807103e+13,11,11,81140,1353.0,213,2,VR05,Twente
4,2020-10-05,2020-10-11,2020,41,Renkum,GM0274,2.216738e+13,1,0,31419,685.0,96,2,VR07,Gelderland-Midden
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32027,2022-06-27,2022-07-03,2022,26,Gennep,GM0907,9.226679e+13,1,0,16921,373.0,32,0,VR23,Limburg-Noord
32028,2022-06-27,2022-07-03,2022,26,Eijsden-Margraten,GM1903,9.874191e+13,0,0,25768,335.0,46,0,VR24,Limburg-Zuid
32029,2022-06-27,2022-07-03,2022,26,Tholen,GM0716,1.062250e+14,0,0,25757,183.0,27,0,VR19,Zeeland
32030,2022-06-27,2022-07-03,2022,26,Dalfsen,GM0148,1.178476e+14,1,1,28587,179.0,49,1,VR04,IJsselland


# Normalized data

In [80]:
# normalized all cases (cases/100000 inhabitants)
final_df['nm_hospital_admission_notification'] = (final_df['Hospital_admission_notification']/final_df['Population'])*100000
final_df['nm_hospital_admission'] = (final_df['Hospital_admission']/final_df['Population'])*100000
final_df['nm_total_reported'] = (final_df['Total_reported']/final_df['Population'])*100000
final_df['nm_deceased'] = (final_df['Deceased']/final_df['Population'])*100000

In [81]:
# fill -9999 value
final_df['nm_hospital_admission_notification'] = np.where(final_df['nm_hospital_admission_notification']<0, -9999, final_df['nm_hospital_admission_notification'])
final_df['nm_hospital_admission'] = np.where(final_df['nm_hospital_admission']<0, -9999, final_df['nm_hospital_admission'])
final_df['nm_total_reported'] = np.where(final_df['nm_total_reported']<0, -9999, final_df['nm_total_reported'])
final_df['nm_deceased'] = np.where(final_df['nm_deceased']<0, -9999, final_df['nm_deceased'])

In [82]:
final_df[final_df['nm_total_reported']<0]

,Start_date,End_date,Year,Week,Municipality_name,Municipality_code,RNA_flow_per_100000_weeklymean,Hospital_admission_notification,Hospital_admission,Population,popdense,Total_reported,Deceased,Security_region_code,Security_region_name,nm_hospital_admission_notification,nm_hospital_admission,nm_total_reported,nm_deceased
57,2020-10-05,2020-10-11,2020,41,Heerhugowaard,GM0398,2.023200e+13,-9999,-9999,57587,NaN,-9999,-9999,NaN,NaN,-9999.0,-9999.0,-9999.0,-9999.0
67,2020-10-05,2020-10-11,2020,41,Uden,GM0856,2.210433e+13,-9999,-9999,42119,NaN,-9999,-9999,NaN,NaN,-9999.0,-9999.0,-9999.0,-9999.0
134,2020-10-05,2020-10-11,2020,41,Landerd,GM1685,1.514705e+14,-9999,-9999,15730,NaN,-9999,-9999,NaN,NaN,-9999.0,-9999.0,-9999.0,-9999.0
150,2020-10-05,2020-10-11,2020,41,Langedijk,GM0416,2.023200e+13,-9999,-9999,28163,NaN,-9999,-9999,NaN,NaN,-9999.0,-9999.0,-9999.0,-9999.0
161,2020-10-05,2020-10-11,2020,41,Weesp,GM0457,2.355709e+13,-9999,-9999,19738,NaN,-9999,-9999,NaN,NaN,-9999.0,-9999.0,-9999.0,-9999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31953,2022-06-27,2022-07-03,2022,26,Boxmeer,GM0756,-9.999000e+03,-9999,-9999,29365,NaN,-9999,-9999,NaN,NaN,-9999.0,-9999.0,-9999.0,-9999.0
31978,2022-06-27,2022-07-03,2022,26,Sint Anthonis,GM1702,-9.999000e+03,-9999,-9999,11664,NaN,-9999,-9999,NaN,NaN,-9999.0,-9999.0,-9999.0,-9999.0
31982,2022-06-27,2022-07-03,2022,26,Cuijk,GM1684,-9.999000e+03,-9999,-9999,25130,NaN,-9999,-9999,NaN,NaN,-9999.0,-9999.0,-9999.0,-9999.0
31989,2022-06-27,2022-07-03,2022,26,Beemster,GM0370,-9.999000e+03,-9999,-9999,10022,NaN,-9999,-9999,NaN,NaN,-9999.0,-9999.0,-9999.0,-9999.0


In [83]:
# final_df.to_csv("data/covid19_2020_2022.csv", encoding='utf-8', index=False)

---

# Accumulated data (placeholder)

In [ ]:
hospitalization_data = hospitalization_data[~((hospitalization_data['Year']==2020)&(hospitalization_data['Week']<41))]

In [ ]:
hospitalization_data

In [ ]:
reported_case_data = reported_case_data[~((reported_case_data['Year']==2020)&(reported_case_data['Week']<41))]

In [ ]:
reported_case_data

In [ ]:
agg_hospitalization_data = hospitalization_data.groupby(['Year', 'Week'])['Hospital_admission'].sum().reset_index()

In [ ]:
agg_reported_case_data = reported_case_data.groupby(['Year', 'Week'])['Total_reported', 'Deceased'].sum().reset_index()

In [ ]:
agg_hospitalization_data

In [ ]:
agg_reported_case_data

In [ ]:
nl_weekly_df = pd.merge(agg_reported_case_data, agg_hospitalization_data, how="inner", on=['Year', 'Week'])

In [ ]:
nl_weekly_df.to_csv("data/covid19_2020_2022_nl.csv", encoding='utf-8', index=False)

# Behaviour data

In [84]:
behavior_data = pd.read_csv('data/COVID-19_gedrag.csv', delimiter=';')

In [85]:
# select record that has 'Totaal' value in 'Subgroup' column and value in 'Region_code' shouldn't be NL00
behavior_total_data = behavior_data[(behavior_data['Subgroup'] == "Totaal") & (behavior_data['Region_code'] != "NL00")]

In [86]:
behavior_total_data = behavior_total_data.drop(columns=['Date_of_report', 'Wave', 'Subgroup_category', 'Subgroup'])

In [87]:
behavior_total_data['Year'] = pd.to_datetime(behavior_total_data['Date_of_measurement']).dt.year
behavior_total_data['Week'] = pd.to_datetime(behavior_total_data['Date_of_measurement']).dt.isocalendar().week

In [88]:
behavior_total_data

,Date_of_measurement,Region_code,Region_name,Indicator_category,Indicator,Sample_size,Figure_type,Value,Lower_limit,Upper_limit,Change_wrt_previous_measurement,Year,Week
11,2020-09-29,VR01,Groningen,Draagvlak,Bij_klachten_blijf_thuis,211,Percentage,85.3,79.7,89.6,NaN,2020,40
12,2020-09-29,VR02,Fryslân,Draagvlak,Bij_klachten_blijf_thuis,192,Percentage,78.8,72.3,84.1,NaN,2020,40
13,2020-09-29,VR03,Drenthe,Draagvlak,Bij_klachten_blijf_thuis,198,Percentage,78.4,71.8,83.8,NaN,2020,40
14,2020-09-29,VR04,IJsselland,Draagvlak,Bij_klachten_blijf_thuis,215,Percentage,85.9,80.6,90.0,NaN,2020,40
15,2020-09-29,VR05,Twente,Draagvlak,Bij_klachten_blijf_thuis,214,Percentage,80.7,74.8,85.5,NaN,2020,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59686,2022-06-21,VR21,Brabant-Noord,Zorgen,Zorgen_over_Coronavirus,220,Percentage,33.8,27.8,40.4,0.0,2022,25
59687,2022-06-21,VR22,Brabant-Zuidoost,Zorgen,Zorgen_over_Coronavirus,212,Percentage,26.8,21.3,33.2,0.0,2022,25
59688,2022-06-21,VR23,Limburg-Noord,Zorgen,Zorgen_over_Coronavirus,237,Percentage,33.5,27.7,39.8,1.0,2022,25
59689,2022-06-21,VR24,Limburg-Zuid,Zorgen,Zorgen_over_Coronavirus,221,Percentage,36.2,30.1,42.8,0.0,2022,25


In [89]:
behavior_total_data[(behavior_total_data['Year']==2022)&(behavior_total_data['Week']==52)]

,Date_of_measurement,Region_code,Region_name,Indicator_category,Indicator,Sample_size,Figure_type,Value,Lower_limit,Upper_limit,Change_wrt_previous_measurement,Year,Week


In [90]:
mun_with_region = final_df[['Municipality_name', 'Municipality_code', 'Security_region_code', 'Security_region_name']].drop_duplicates()

In [91]:
# some municipalities has no information about their safety region
mun_with_region[mun_with_region['Security_region_code'].isna()]

,Municipality_name,Municipality_code,Security_region_code,Security_region_name
57,Heerhugowaard,GM0398,NaN,NaN
67,Uden,GM0856,NaN,NaN
134,Landerd,GM1685,NaN,NaN
150,Langedijk,GM0416,NaN,NaN
161,Weesp,GM0457,NaN,NaN
227,Mill en Sint Hubert,GM0815,NaN,NaN
273,Boxmeer,GM0756,NaN,NaN
298,Sint Anthonis,GM1702,NaN,NaN
302,Cuijk,GM1684,NaN,NaN
309,Beemster,GM0370,NaN,NaN


Municipality and safety region information
- Heerhugowaard - Noord-Holland-Noord VR10
- Uden - North Brabant (Brabant-Noord) VR21
- Landerd - North Brabant (Brabant-Noord) VR21
- Langedijk - Noord-Holland-Noord VR10
- Weesp - Gooi and Vechtstreek (Gooi en Vechtstreek) VR14
- Mill en Sint Hubert - North Brabant (Brabant-Noord) VR21
- Boxmeer - North Brabant (Brabant-Noord) VR21
- Sint Anthonis - North Brabant (Brabant-Noord) VR21
- Cuijk - North Brabant (Brabant-Noord) VR21
- Beemster - Zaanstreek-Waterland VR11
- Grave - North Brabant (Brabant-Noord) VR21
- Hellevoetsluis - Rotterdam-Rijnmond VR17
- Brielle - Rotterdam-Rijnmond VR17
- Westvoorne - Rotterdam-Rijnmond VR17

source: https://wetten.overheid.nl/BWBR0027466/2019-01-01

In [92]:
security_reg_code = ["VR10", "VR21", "VR21", "VR10", "VR14", "VR21", "VR21", "VR21", "VR21", "VR11", "VR21", "VR17", "VR17", "VR17"]
security_reg_name = ["Noord-Holland-Noord", "Brabant-Noord", "Brabant-Noord", "Noord-Holland-Noord", "Gooi en Vechtstreek", "Brabant-Noord", "Brabant-Noord", "Brabant-Noord", "Brabant-Noord", "Zaanstreek-Waterland", "Brabant-Noord", "Rotterdam-Rijnmond", "Rotterdam-Rijnmond", "Rotterdam-Rijnmond"]

In [93]:
# put safety region information
mun_with_region.loc[mun_with_region['Security_region_code'].isna(), 'Security_region_code'] = security_reg_code
mun_with_region.loc[mun_with_region['Security_region_name'].isna(), 'Security_region_name'] = security_reg_name

In [94]:
mun_with_region[mun_with_region['Municipality_name'] == "Weesp"]

,Municipality_name,Municipality_code,Security_region_code,Security_region_name
161,Weesp,GM0457,VR14,Gooi en Vechtstreek


In [95]:
mun_with_region

,Municipality_name,Municipality_code,Security_region_code,Security_region_name
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland
1,Valkenswaard,GM0858,VR22,Brabant-Zuidoost
2,Wierden,GM0189,VR05,Twente
3,Hengelo (O),GM0164,VR05,Twente
4,Renkum,GM0274,VR07,Gelderland-Midden
...,...,...,...,...
350,Dalfsen,GM0148,VR04,IJsselland
351,Hof van Twente,GM1735,VR05,Twente
18474,Hellevoetsluis,GM0530,VR17,Rotterdam-Rijnmond
18555,Brielle,GM0501,VR17,Rotterdam-Rijnmond


In [96]:
mun_with_region = mun_with_region.rename({'Security_region_code': 'Region_code', 'Security_region_name': 'Region_name'}, axis=1)

In [97]:
mun_with_region

,Municipality_name,Municipality_code,Region_code,Region_name
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland
1,Valkenswaard,GM0858,VR22,Brabant-Zuidoost
2,Wierden,GM0189,VR05,Twente
3,Hengelo (O),GM0164,VR05,Twente
4,Renkum,GM0274,VR07,Gelderland-Midden
...,...,...,...,...
350,Dalfsen,GM0148,VR04,IJsselland
351,Hof van Twente,GM1735,VR05,Twente
18474,Hellevoetsluis,GM0530,VR17,Rotterdam-Rijnmond
18555,Brielle,GM0501,VR17,Rotterdam-Rijnmond


In [98]:
mun_with_region.loc[mun_with_region['Region_name'].isna(), 'Region_name']

Series([], Name: Region_name, dtype: object)

In [99]:
behavior_total_data = behavior_total_data.reset_index()

In [100]:
behavior_total_data = behavior_total_data.drop(columns=['index'])

In [101]:
# behavior data is collected 3-4 weeks and to extent it to weekly scale, I need to determine the time range of the data 
# we already have sorted data so the logic is if the record shows new date that date is the end date of previous record (use week number to determine start and end of time range of data)
# e.g. current record -> Year 2020 Week 44 (End week is Week 47)
#      next record -> Year 2020 Week 47
for i in range(len(behavior_total_data)):
    region_code = behavior_total_data.loc[i, "Region_code"]
    year = behavior_total_data.loc[i, "Year"]
    week = behavior_total_data.loc[i, "Week"]
    temp_df = behavior_total_data[(behavior_total_data['Week']>week) & (behavior_total_data['Year']==year) & (behavior_total_data['Region_code']==region_code)]
    if(temp_df.empty):
        if((year == 2022) and (week == 25)):
            end_week = 26
        else:
            temp_df = behavior_total_data[(behavior_total_data['Week']==1) & (behavior_total_data['Year']==year+1) & (behavior_total_data['Region_code']==region_code)]
            next_loc = temp_df.index[0]
            end_week = behavior_total_data.loc[next_loc, "Week"]
    else:
        next_loc = temp_df.index[0]
        end_week = behavior_total_data.loc[next_loc, "Week"]
    # print(region_code)
    # print(final_behavior_data.loc[next_loc, "Week"])
    behavior_total_data.loc[i, "End_week"] = end_week
    

In [102]:
behavior_total_data['End_week'] = behavior_total_data['End_week'].astype(int)
behavior_total_data

,Date_of_measurement,Region_code,Region_name,Indicator_category,Indicator,Sample_size,Figure_type,Value,Lower_limit,Upper_limit,Change_wrt_previous_measurement,Year,Week,End_week
0,2020-09-29,VR01,Groningen,Draagvlak,Bij_klachten_blijf_thuis,211,Percentage,85.3,79.7,89.6,NaN,2020,40,43
1,2020-09-29,VR02,Fryslân,Draagvlak,Bij_klachten_blijf_thuis,192,Percentage,78.8,72.3,84.1,NaN,2020,40,43
2,2020-09-29,VR03,Drenthe,Draagvlak,Bij_klachten_blijf_thuis,198,Percentage,78.4,71.8,83.8,NaN,2020,40,43
3,2020-09-29,VR04,IJsselland,Draagvlak,Bij_klachten_blijf_thuis,215,Percentage,85.9,80.6,90.0,NaN,2020,40,43
4,2020-09-29,VR05,Twente,Draagvlak,Bij_klachten_blijf_thuis,214,Percentage,80.7,74.8,85.5,NaN,2020,40,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41429,2022-06-21,VR21,Brabant-Noord,Zorgen,Zorgen_over_Coronavirus,220,Percentage,33.8,27.8,40.4,0.0,2022,25,26
41430,2022-06-21,VR22,Brabant-Zuidoost,Zorgen,Zorgen_over_Coronavirus,212,Percentage,26.8,21.3,33.2,0.0,2022,25,26
41431,2022-06-21,VR23,Limburg-Noord,Zorgen,Zorgen_over_Coronavirus,237,Percentage,33.5,27.7,39.8,1.0,2022,25,26
41432,2022-06-21,VR24,Limburg-Zuid,Zorgen,Zorgen_over_Coronavirus,221,Percentage,36.2,30.1,42.8,0.0,2022,25,26


In [103]:
# create running number of each pair of 'start and end week'
flag = 0
previous_week = 40
start_week_list = []
end_week_list = []
flag_list = []
count = 0
for i in range(len(behavior_total_data)):
    week = behavior_total_data.loc[i, "Week"]
    if(week != previous_week):
        flag += 1
        count = 0
    behavior_total_data.loc[i, 'flag'] = flag
    if(count==0):
        start_week_list.append(week)
        end_week_list.append(behavior_total_data.loc[i, "End_week"])
        flag_list.append(behavior_total_data.loc[i, "flag"])
        count += 1
    previous_week = behavior_total_data.loc[i, "Week"]

In [104]:
behavior_total_data

,Date_of_measurement,Region_code,Region_name,Indicator_category,Indicator,Sample_size,Figure_type,Value,Lower_limit,Upper_limit,Change_wrt_previous_measurement,Year,Week,End_week,flag
0,2020-09-29,VR01,Groningen,Draagvlak,Bij_klachten_blijf_thuis,211,Percentage,85.3,79.7,89.6,NaN,2020,40,43,0.0
1,2020-09-29,VR02,Fryslân,Draagvlak,Bij_klachten_blijf_thuis,192,Percentage,78.8,72.3,84.1,NaN,2020,40,43,0.0
2,2020-09-29,VR03,Drenthe,Draagvlak,Bij_klachten_blijf_thuis,198,Percentage,78.4,71.8,83.8,NaN,2020,40,43,0.0
3,2020-09-29,VR04,IJsselland,Draagvlak,Bij_klachten_blijf_thuis,215,Percentage,85.9,80.6,90.0,NaN,2020,40,43,0.0
4,2020-09-29,VR05,Twente,Draagvlak,Bij_klachten_blijf_thuis,214,Percentage,80.7,74.8,85.5,NaN,2020,40,43,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41429,2022-06-21,VR21,Brabant-Noord,Zorgen,Zorgen_over_Coronavirus,220,Percentage,33.8,27.8,40.4,0.0,2022,25,26,29.0
41430,2022-06-21,VR22,Brabant-Zuidoost,Zorgen,Zorgen_over_Coronavirus,212,Percentage,26.8,21.3,33.2,0.0,2022,25,26,29.0
41431,2022-06-21,VR23,Limburg-Noord,Zorgen,Zorgen_over_Coronavirus,237,Percentage,33.5,27.7,39.8,1.0,2022,25,26,29.0
41432,2022-06-21,VR24,Limburg-Zuid,Zorgen,Zorgen_over_Coronavirus,221,Percentage,36.2,30.1,42.8,0.0,2022,25,26,29.0


In [105]:
print(start_week_list)
print(end_week_list)
print(flag_list)

[40, 43, 46, 49, 51, 1, 4, 7, 10, 13, 16, 19, 22, 26, 28, 31, 34, 37, 40, 43, 46, 49, 1, 4, 7, 10, 13, 17, 21, 25]
[43, 46, 49, 51, 1, 4, 7, 10, 13, 16, 19, 22, 26, 28, 31, 34, 37, 40, 43, 46, 49, 1, 4, 7, 10, 13, 17, 21, 25, 26]
[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0]


In [106]:
# assign running number to initial date dataframe so that we can merge initial date dataframe with behavior data
change = 0
for i in range(len(date_df)):
    week = date_df.loc[i, "Week"]
    for j in range(len(start_week_list)):
        if(((week>=start_week_list[j]) and (week<end_week_list[j])) or 
           ((week>=start_week_list[j]) and (end_week_list[j]==1)) or
           ((week==26) and (start_week_list[j]==25) and (end_week_list[j]==26))):
            date_df.loc[i, 'flag'] = flag_list[j]
            if(week==start_week_list[j]):
                start_week_list.pop(j-1)
                end_week_list.pop(j-1)
                flag_list.pop(j-1)
            break

In [107]:
date_df['flag'] = date_df['flag'].astype(int)
date_df

,Start_date,End_date,Year,Week,flag
0,2020-10-05,2020-10-11,2020,41,0
1,2020-10-12,2020-10-18,2020,42,0
2,2020-10-19,2020-10-25,2020,43,1
3,2020-10-26,2020-11-01,2020,44,1
4,2020-11-02,2020-11-08,2020,45,1
...,...,...,...,...,...
86,2022-05-30,2022-06-05,2022,22,28
87,2022-06-06,2022-06-12,2022,23,28
88,2022-06-13,2022-06-19,2022,24,28
89,2022-06-20,2022-06-26,2022,25,29


In [108]:
date_df[date_df['Year']==2020]

,Start_date,End_date,Year,Week,flag
0,2020-10-05,2020-10-11,2020,41,0
1,2020-10-12,2020-10-18,2020,42,0
2,2020-10-19,2020-10-25,2020,43,1
3,2020-10-26,2020-11-01,2020,44,1
4,2020-11-02,2020-11-08,2020,45,1
5,2020-11-09,2020-11-15,2020,46,2
6,2020-11-16,2020-11-22,2020,47,2
7,2020-11-23,2020-11-29,2020,48,2
8,2020-11-30,2020-12-06,2020,49,3
9,2020-12-07,2020-12-13,2020,50,3


In [109]:
# join two dataframes (initial date dataframe and behavior data)
# use .join() instead of .merge() because .join() has better performance that .merge()
test_df = date_df.join(behavior_total_data.set_index('flag'), on='flag', how = "outer", lsuffix='_left', rsuffix='_right')

In [110]:
test_df

,Start_date,End_date,Year_left,Week_left,flag,Date_of_measurement,Region_code,Region_name,Indicator_category,Indicator,Sample_size,Figure_type,Value,Lower_limit,Upper_limit,Change_wrt_previous_measurement,Year_right,Week_right,End_week
0,2020-10-05,2020-10-11,2020,41,0,2020-09-29,VR01,Groningen,Draagvlak,Bij_klachten_blijf_thuis,211,Percentage,85.3,79.7,89.6,NaN,2020,40,43
0,2020-10-05,2020-10-11,2020,41,0,2020-09-29,VR02,Fryslân,Draagvlak,Bij_klachten_blijf_thuis,192,Percentage,78.8,72.3,84.1,NaN,2020,40,43
0,2020-10-05,2020-10-11,2020,41,0,2020-09-29,VR03,Drenthe,Draagvlak,Bij_klachten_blijf_thuis,198,Percentage,78.4,71.8,83.8,NaN,2020,40,43
0,2020-10-05,2020-10-11,2020,41,0,2020-09-29,VR04,IJsselland,Draagvlak,Bij_klachten_blijf_thuis,215,Percentage,85.9,80.6,90.0,NaN,2020,40,43
0,2020-10-05,2020-10-11,2020,41,0,2020-09-29,VR05,Twente,Draagvlak,Bij_klachten_blijf_thuis,214,Percentage,80.7,74.8,85.5,NaN,2020,40,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,2022-06-27,2022-07-03,2022,26,29,2022-06-21,VR21,Brabant-Noord,Zorgen,Zorgen_over_Coronavirus,220,Percentage,33.8,27.8,40.4,0.0,2022,25,26
90,2022-06-27,2022-07-03,2022,26,29,2022-06-21,VR22,Brabant-Zuidoost,Zorgen,Zorgen_over_Coronavirus,212,Percentage,26.8,21.3,33.2,0.0,2022,25,26
90,2022-06-27,2022-07-03,2022,26,29,2022-06-21,VR23,Limburg-Noord,Zorgen,Zorgen_over_Coronavirus,237,Percentage,33.5,27.7,39.8,1.0,2022,25,26
90,2022-06-27,2022-07-03,2022,26,29,2022-06-21,VR24,Limburg-Zuid,Zorgen,Zorgen_over_Coronavirus,221,Percentage,36.2,30.1,42.8,0.0,2022,25,26


In [111]:
# make it into municipality level
final_behavior_data = mun_with_region.join(test_df.set_index('Region_code'), on='Region_code', how = "outer", lsuffix='_left', rsuffix='_right')

In [112]:
final_behavior_data

,Municipality_name,Municipality_code,Region_code,Region_name_left,Start_date,End_date,Year_left,Week_left,flag,Date_of_measurement,...,Indicator,Sample_size,Figure_type,Value,Lower_limit,Upper_limit,Change_wrt_previous_measurement,Year_right,Week_right,End_week
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,...,Bij_klachten_blijf_thuis,193,Percentage,83.3,77.1,88.1,NaN,2020,40,43
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,...,Bij_klachten_laat_testen,192,Percentage,61.5,53.8,68.7,NaN,2020,40,43
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,...,Houd_1_5m_afstand,193,Percentage,92.6,88.2,95.5,NaN,2020,40,43
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,...,Ontvang_max_bezoekers_thuis,192,Percentage,77.0,70.1,82.8,NaN,2020,40,43
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,...,Vermijd_drukke_plekken,193,Percentage,95.5,91.0,97.8,NaN,2020,40,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,...,Thuisgewerkte_uren,36,Percentage,43.6,33.0,54.3,0.0,2022,25,26
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,...,Ventileren_woning,117,Percentage,57.5,47.8,66.6,0.0,2022,25,26
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,...,Was_vaak_je_handen,117,Percentage,34.9,26.3,44.5,0.0,2022,25,26
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,...,Werkt_thuis,59,Percentage,70.6,57.5,81.0,0.0,2022,25,26


In [113]:
final_behavior_data.columns

Index(['Municipality_name', 'Municipality_code', 'Region_code',
       'Region_name_left', 'Start_date', 'End_date', 'Year_left', 'Week_left',
       'flag', 'Date_of_measurement', 'Region_name_right',
       'Indicator_category', 'Indicator', 'Sample_size', 'Figure_type',
       'Value', 'Lower_limit', 'Upper_limit',
       'Change_wrt_previous_measurement', 'Year_right', 'Week_right',
       'End_week'],
      dtype='object')

In [114]:
final_behavior_data = final_behavior_data.drop(columns=['Region_name_right', 'Year_right', 'Week_right', 'End_week'])
final_behavior_data = final_behavior_data.rename({'Region_name_left': 'Region_name', 'CODE': 'Municipality_code', 'Year_left': 'Year', 'Week_left': 'Week'}, axis=1)

In [115]:
final_behavior_data

,Municipality_name,Municipality_code,Region_code,Region_name,Start_date,End_date,Year,Week,flag,Date_of_measurement,Indicator_category,Indicator,Sample_size,Figure_type,Value,Lower_limit,Upper_limit,Change_wrt_previous_measurement
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Bij_klachten_blijf_thuis,193,Percentage,83.3,77.1,88.1,NaN
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Bij_klachten_laat_testen,192,Percentage,61.5,53.8,68.7,NaN
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Houd_1_5m_afstand,193,Percentage,92.6,88.2,95.5,NaN
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Ontvang_max_bezoekers_thuis,192,Percentage,77.0,70.1,82.8,NaN
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Vermijd_drukke_plekken,193,Percentage,95.5,91.0,97.8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,Naleving,Thuisgewerkte_uren,36,Percentage,43.6,33.0,54.3,0.0
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,Naleving,Ventileren_woning,117,Percentage,57.5,47.8,66.6,0.0
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,Naleving,Was_vaak_je_handen,117,Percentage,34.9,26.3,44.5,0.0
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,Naleving,Werkt_thuis,59,Percentage,70.6,57.5,81.0,0.0


In [116]:
# check number of safety region
final_behavior_data['Region_name'].nunique()

25

In [117]:
# final_behavior_data = pd.merge(mun_with_region, behavior_total_data, how="outer", on=['Region_code'])

In [118]:
# final_behavior_data.to_csv("data/covid19_behavior_data.csv", encoding='utf-8', index=False)

# Merged behavior data

In [119]:
final_behavior_data

,Municipality_name,Municipality_code,Region_code,Region_name,Start_date,End_date,Year,Week,flag,Date_of_measurement,Indicator_category,Indicator,Sample_size,Figure_type,Value,Lower_limit,Upper_limit,Change_wrt_previous_measurement
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Bij_klachten_blijf_thuis,193,Percentage,83.3,77.1,88.1,NaN
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Bij_klachten_laat_testen,192,Percentage,61.5,53.8,68.7,NaN
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Houd_1_5m_afstand,193,Percentage,92.6,88.2,95.5,NaN
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Ontvang_max_bezoekers_thuis,192,Percentage,77.0,70.1,82.8,NaN
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Vermijd_drukke_plekken,193,Percentage,95.5,91.0,97.8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,Naleving,Thuisgewerkte_uren,36,Percentage,43.6,33.0,54.3,0.0
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,Naleving,Ventileren_woning,117,Percentage,57.5,47.8,66.6,0.0
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,Naleving,Was_vaak_je_handen,117,Percentage,34.9,26.3,44.5,0.0
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,Naleving,Werkt_thuis,59,Percentage,70.6,57.5,81.0,0.0


In [120]:
merge_behavior_data = final_behavior_data.drop(columns=['Sample_size', 'Figure_type', 'Lower_limit', 'Upper_limit', 'Change_wrt_previous_measurement'])

In [121]:
merge_behavior_data

,Municipality_name,Municipality_code,Region_code,Region_name,Start_date,End_date,Year,Week,flag,Date_of_measurement,Indicator_category,Indicator,Value
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Bij_klachten_blijf_thuis,83.3
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Bij_klachten_laat_testen,61.5
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Houd_1_5m_afstand,92.6
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Ontvang_max_bezoekers_thuis,77.0
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Vermijd_drukke_plekken,95.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,Naleving,Thuisgewerkte_uren,43.6
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,Naleving,Ventileren_woning,57.5
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,Naleving,Was_vaak_je_handen,34.9
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,Naleving,Werkt_thuis,70.6


In [122]:
# get all possible values in 'Indicator' column
merge_behavior_data['Indicator'].unique()

array(['Bij_klachten_blijf_thuis', 'Bij_klachten_laat_testen',
       'Houd_1_5m_afstand', 'Ontvang_max_bezoekers_thuis',
       'Vermijd_drukke_plekken', 'Was_vaak_je_handen', 'Werkt_thuis',
       'Ja', 'Nee', 'Weet_niet', 'Minstens_een_corona_gerelateerd',
       'Al_gevaccineerd', 'Zorgen_over_Coronavirus',
       'Draag_mondkapje_in_ov',
       'Draag_mondkapje_in_publieke_binnenruimtes',
       'Hoest_niest_in_elleboog', 'Thuisgewerkte_uren', 'Avondklok',
       'Bij_klachten_blijf_terecht_thuis', 'Ventileren_woning',
       'Zelftest_bezoek', 'Draag_mondkapje_op_drukke_plekken',
       'Bij_klachten_postest_isolatie'], dtype=object)

In [123]:
# select only potential behavior data
merge_behavior_data = merge_behavior_data[(merge_behavior_data['Indicator']=="Avondklok")|
                                          (merge_behavior_data['Indicator']=="Draag_mondkapje_in_ov")|
                                          (merge_behavior_data['Indicator']=="Draag_mondkapje_in_publieke_binnenruimtes")|
                                          (merge_behavior_data['Indicator']=="Draag_mondkapje_op_drukke_plekken")|
                                          (merge_behavior_data['Indicator']=="Hoest_niest_in_elleboog")|
                                          (merge_behavior_data['Indicator']=="Houd_1_5m_afstand")|
                                          (merge_behavior_data['Indicator']=="Thuisgewerkte_uren")|
                                          (merge_behavior_data['Indicator']=="Ventileren_woning")|
                                          (merge_behavior_data['Indicator']=="Vermijd_drukke_plekken")|
                                          (merge_behavior_data['Indicator']=="Was_vaak_je_handen")|
                                          (merge_behavior_data['Indicator']=="Werkt_thuis")|
                                          (merge_behavior_data['Indicator']=="Zelftest_bezoek")|
                                          (merge_behavior_data['Indicator']=="Zorgen_over_Coronavirus")|
                                          (merge_behavior_data['Indicator']=="Al_gevaccineerd")|
                                          (merge_behavior_data['Indicator']=="Minstens_een_corona_gerelateerd")|
                                          (merge_behavior_data['Indicator']=="Zelftest_bezoek")|
                                          (merge_behavior_data['Indicator']=="Bij_klachten_postest_isolatie")]

In [124]:
merge_behavior_data

,Municipality_name,Municipality_code,Region_code,Region_name,Start_date,End_date,Year,Week,flag,Date_of_measurement,Indicator_category,Indicator,Value
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Houd_1_5m_afstand,92.6
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Vermijd_drukke_plekken,95.5
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Was_vaak_je_handen,88.7
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Draagvlak,Werkt_thuis,78.3
0,Nunspeet,GM0302,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,2020,41,0,2020-09-29,Helpen_regels,Houd_1_5m_afstand,83.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,Naleving,Thuisgewerkte_uren,43.6
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,Naleving,Ventileren_woning,57.5
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,Naleving,Was_vaak_je_handen,34.9
313,Blaricum,GM0376,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,2022,26,29,2022-06-21,Naleving,Werkt_thuis,70.6


In [125]:
final_df

,Start_date,End_date,Year,Week,Municipality_name,Municipality_code,RNA_flow_per_100000_weeklymean,Hospital_admission_notification,Hospital_admission,Population,popdense,Total_reported,Deceased,Security_region_code,Security_region_name,nm_hospital_admission_notification,nm_hospital_admission,nm_total_reported,nm_deceased
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,9.017750e+12,3,0,27851,223.0,40,0,VR06,Noord- en Oost-Gelderland,10.771606,0.000000,143.621414,0.000000
1,2020-10-05,2020-10-11,2020,41,Valkenswaard,GM0858,-9.999000e+03,0,1,31193,574.0,68,0,VR22,Brabant-Zuidoost,0.000000,3.205847,217.997628,0.000000
2,2020-10-05,2020-10-11,2020,41,Wierden,GM0189,2.173360e+13,2,2,24446,263.0,45,0,VR05,Twente,8.181298,8.181298,184.079195,0.000000
3,2020-10-05,2020-10-11,2020,41,Hengelo (O),GM0164,4.807103e+13,11,11,81140,1353.0,213,2,VR05,Twente,13.556815,13.556815,262.509243,2.464876
4,2020-10-05,2020-10-11,2020,41,Renkum,GM0274,2.216738e+13,1,0,31419,685.0,96,2,VR07,Gelderland-Midden,3.182787,0.000000,305.547599,6.365575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32027,2022-06-27,2022-07-03,2022,26,Gennep,GM0907,9.226679e+13,1,0,16921,373.0,32,0,VR23,Limburg-Noord,5.909816,0.000000,189.114119,0.000000
32028,2022-06-27,2022-07-03,2022,26,Eijsden-Margraten,GM1903,9.874191e+13,0,0,25768,335.0,46,0,VR24,Limburg-Zuid,0.000000,0.000000,178.515989,0.000000
32029,2022-06-27,2022-07-03,2022,26,Tholen,GM0716,1.062250e+14,0,0,25757,183.0,27,0,VR19,Zeeland,0.000000,0.000000,104.825873,0.000000
32030,2022-06-27,2022-07-03,2022,26,Dalfsen,GM0148,1.178476e+14,1,1,28587,179.0,49,1,VR04,IJsselland,3.498094,3.498094,171.406583,3.498094


In [126]:
# join the previous dataframe that contain wastewater viral load, reported case, mortality case, and hospitalization case and behavior data dataframe together
new_final_df = final_df.join(merge_behavior_data.set_index(['Year', 'Week', 'Municipality_code']), on=['Year', 'Week', 'Municipality_code'], how = "right", lsuffix='_left', rsuffix='_right')

In [127]:
new_final_df

,Start_date_left,End_date_left,Year,Week,Municipality_name_left,Municipality_code,RNA_flow_per_100000_weeklymean,Hospital_admission_notification,Hospital_admission,Population,...,Municipality_name_right,Region_code,Region_name,Start_date_right,End_date_right,flag,Date_of_measurement,Indicator_category,Indicator,Value
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,9.017750e+12,3,0,27851,...,Nunspeet,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,0,2020-09-29,Draagvlak,Houd_1_5m_afstand,92.6
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,9.017750e+12,3,0,27851,...,Nunspeet,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,0,2020-09-29,Draagvlak,Vermijd_drukke_plekken,95.5
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,9.017750e+12,3,0,27851,...,Nunspeet,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,0,2020-09-29,Draagvlak,Was_vaak_je_handen,88.7
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,9.017750e+12,3,0,27851,...,Nunspeet,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,0,2020-09-29,Draagvlak,Werkt_thuis,78.3
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,9.017750e+12,3,0,27851,...,Nunspeet,VR06,Noord- en Oost-Gelderland,2020-10-05,2020-10-11,0,2020-09-29,Helpen_regels,Houd_1_5m_afstand,83.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31993,2022-06-27,2022-07-03,2022,26,Blaricum,GM0376,1.025120e+14,1,0,11540,...,Blaricum,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,29,2022-06-21,Naleving,Thuisgewerkte_uren,43.6
31993,2022-06-27,2022-07-03,2022,26,Blaricum,GM0376,1.025120e+14,1,0,11540,...,Blaricum,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,29,2022-06-21,Naleving,Ventileren_woning,57.5
31993,2022-06-27,2022-07-03,2022,26,Blaricum,GM0376,1.025120e+14,1,0,11540,...,Blaricum,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,29,2022-06-21,Naleving,Was_vaak_je_handen,34.9
31993,2022-06-27,2022-07-03,2022,26,Blaricum,GM0376,1.025120e+14,1,0,11540,...,Blaricum,VR14,Gooi en Vechtstreek,2022-06-27,2022-07-03,29,2022-06-21,Naleving,Werkt_thuis,70.6


In [128]:
new_final_df.columns

Index(['Start_date_left', 'End_date_left', 'Year', 'Week',
       'Municipality_name_left', 'Municipality_code',
       'RNA_flow_per_100000_weeklymean', 'Hospital_admission_notification',
       'Hospital_admission', 'Population', 'popdense', 'Total_reported',
       'Deceased', 'Security_region_code', 'Security_region_name',
       'nm_hospital_admission_notification', 'nm_hospital_admission',
       'nm_total_reported', 'nm_deceased', 'Municipality_name_right',
       'Region_code', 'Region_name', 'Start_date_right', 'End_date_right',
       'flag', 'Date_of_measurement', 'Indicator_category', 'Indicator',
       'Value'],
      dtype='object')

In [129]:
new_final_df[new_final_df['Region_code'].isna()]

,Start_date_left,End_date_left,Year,Week,Municipality_name_left,Municipality_code,RNA_flow_per_100000_weeklymean,Hospital_admission_notification,Hospital_admission,Population,...,Municipality_name_right,Region_code,Region_name,Start_date_right,End_date_right,flag,Date_of_measurement,Indicator_category,Indicator,Value


In [130]:
new_final_df[new_final_df['Municipality_name_left']=="Nunspeet"][['Security_region_code', 'Security_region_name', 'Region_code', 'Region_name']]

,Security_region_code,Security_region_name,Region_code,Region_name
0,VR06,Noord- en Oost-Gelderland,VR06,Noord- en Oost-Gelderland
0,VR06,Noord- en Oost-Gelderland,VR06,Noord- en Oost-Gelderland
0,VR06,Noord- en Oost-Gelderland,VR06,Noord- en Oost-Gelderland
0,VR06,Noord- en Oost-Gelderland,VR06,Noord- en Oost-Gelderland
0,VR06,Noord- en Oost-Gelderland,VR06,Noord- en Oost-Gelderland
...,...,...,...,...
31680,VR06,Noord- en Oost-Gelderland,VR06,Noord- en Oost-Gelderland
31680,VR06,Noord- en Oost-Gelderland,VR06,Noord- en Oost-Gelderland
31680,VR06,Noord- en Oost-Gelderland,VR06,Noord- en Oost-Gelderland
31680,VR06,Noord- en Oost-Gelderland,VR06,Noord- en Oost-Gelderland


In [131]:
new_final_df = new_final_df.drop(columns=['Municipality_name_right', 'Security_region_code', 'Security_region_name', 'Start_date_right', 'End_date_right', 'Date_of_measurement'])

In [132]:
new_final_df

,Start_date_left,End_date_left,Year,Week,Municipality_name_left,Municipality_code,RNA_flow_per_100000_weeklymean,Hospital_admission_notification,Hospital_admission,Population,...,nm_hospital_admission_notification,nm_hospital_admission,nm_total_reported,nm_deceased,Region_code,Region_name,flag,Indicator_category,Indicator,Value
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,9.017750e+12,3,0,27851,...,10.771606,0.0,143.621414,0.0,VR06,Noord- en Oost-Gelderland,0,Draagvlak,Houd_1_5m_afstand,92.6
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,9.017750e+12,3,0,27851,...,10.771606,0.0,143.621414,0.0,VR06,Noord- en Oost-Gelderland,0,Draagvlak,Vermijd_drukke_plekken,95.5
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,9.017750e+12,3,0,27851,...,10.771606,0.0,143.621414,0.0,VR06,Noord- en Oost-Gelderland,0,Draagvlak,Was_vaak_je_handen,88.7
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,9.017750e+12,3,0,27851,...,10.771606,0.0,143.621414,0.0,VR06,Noord- en Oost-Gelderland,0,Draagvlak,Werkt_thuis,78.3
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,9.017750e+12,3,0,27851,...,10.771606,0.0,143.621414,0.0,VR06,Noord- en Oost-Gelderland,0,Helpen_regels,Houd_1_5m_afstand,83.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31993,2022-06-27,2022-07-03,2022,26,Blaricum,GM0376,1.025120e+14,1,0,11540,...,8.665511,0.0,285.961872,0.0,VR14,Gooi en Vechtstreek,29,Naleving,Thuisgewerkte_uren,43.6
31993,2022-06-27,2022-07-03,2022,26,Blaricum,GM0376,1.025120e+14,1,0,11540,...,8.665511,0.0,285.961872,0.0,VR14,Gooi en Vechtstreek,29,Naleving,Ventileren_woning,57.5
31993,2022-06-27,2022-07-03,2022,26,Blaricum,GM0376,1.025120e+14,1,0,11540,...,8.665511,0.0,285.961872,0.0,VR14,Gooi en Vechtstreek,29,Naleving,Was_vaak_je_handen,34.9
31993,2022-06-27,2022-07-03,2022,26,Blaricum,GM0376,1.025120e+14,1,0,11540,...,8.665511,0.0,285.961872,0.0,VR14,Gooi en Vechtstreek,29,Naleving,Werkt_thuis,70.6


In [133]:
new_final_df = new_final_df.rename({"Start_date_left": "Start_date", "End_date_left": "End_date", "Municipality_name_left": "Municipality_name"}, axis=1)

In [134]:
new_final_df

,Start_date,End_date,Year,Week,Municipality_name,Municipality_code,RNA_flow_per_100000_weeklymean,Hospital_admission_notification,Hospital_admission,Population,...,nm_hospital_admission_notification,nm_hospital_admission,nm_total_reported,nm_deceased,Region_code,Region_name,flag,Indicator_category,Indicator,Value
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,9.017750e+12,3,0,27851,...,10.771606,0.0,143.621414,0.0,VR06,Noord- en Oost-Gelderland,0,Draagvlak,Houd_1_5m_afstand,92.6
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,9.017750e+12,3,0,27851,...,10.771606,0.0,143.621414,0.0,VR06,Noord- en Oost-Gelderland,0,Draagvlak,Vermijd_drukke_plekken,95.5
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,9.017750e+12,3,0,27851,...,10.771606,0.0,143.621414,0.0,VR06,Noord- en Oost-Gelderland,0,Draagvlak,Was_vaak_je_handen,88.7
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,9.017750e+12,3,0,27851,...,10.771606,0.0,143.621414,0.0,VR06,Noord- en Oost-Gelderland,0,Draagvlak,Werkt_thuis,78.3
0,2020-10-05,2020-10-11,2020,41,Nunspeet,GM0302,9.017750e+12,3,0,27851,...,10.771606,0.0,143.621414,0.0,VR06,Noord- en Oost-Gelderland,0,Helpen_regels,Houd_1_5m_afstand,83.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31993,2022-06-27,2022-07-03,2022,26,Blaricum,GM0376,1.025120e+14,1,0,11540,...,8.665511,0.0,285.961872,0.0,VR14,Gooi en Vechtstreek,29,Naleving,Thuisgewerkte_uren,43.6
31993,2022-06-27,2022-07-03,2022,26,Blaricum,GM0376,1.025120e+14,1,0,11540,...,8.665511,0.0,285.961872,0.0,VR14,Gooi en Vechtstreek,29,Naleving,Ventileren_woning,57.5
31993,2022-06-27,2022-07-03,2022,26,Blaricum,GM0376,1.025120e+14,1,0,11540,...,8.665511,0.0,285.961872,0.0,VR14,Gooi en Vechtstreek,29,Naleving,Was_vaak_je_handen,34.9
31993,2022-06-27,2022-07-03,2022,26,Blaricum,GM0376,1.025120e+14,1,0,11540,...,8.665511,0.0,285.961872,0.0,VR14,Gooi en Vechtstreek,29,Naleving,Werkt_thuis,70.6


In [135]:
new_final_df.columns

Index(['Start_date', 'End_date', 'Year', 'Week', 'Municipality_name',
       'Municipality_code', 'RNA_flow_per_100000_weeklymean',
       'Hospital_admission_notification', 'Hospital_admission', 'Population',
       'popdense', 'Total_reported', 'Deceased',
       'nm_hospital_admission_notification', 'nm_hospital_admission',
       'nm_total_reported', 'nm_deceased', 'Region_code', 'Region_name',
       'flag', 'Indicator_category', 'Indicator', 'Value'],
      dtype='object')

In [140]:
# write final dataset to csv file
new_final_df.to_csv("data/processed_data/covid19_2020_2022_all.csv", encoding='utf-8', index=False)

____

# Pivot dataframe (placeholder)

In [ ]:
new_final_df = pd.read_csv("data/covid19_2020_2022_all.csv")

In [ ]:
new_final_df

In [ ]:
new_final_df['flag'].unique()

In [ ]:
new_final_df[new_final_df['Indicator_category'].isna()]

In [ ]:
pd.pivot_table(new_final_df, values = 'Value', index=['Year','Week', 'Municipality_name', 'Municipality_code'], columns = ['Indicator_category', 'Indicator']).reset_index()


In [ ]:
compliance_df = new_final_df[new_final_df['Indicator_category'] == "Naleving"]

In [ ]:
compliance_df

In [ ]:
pivot_compliance_df = pd.pivot_table(compliance_df, values = 'Value', index=['Start_date', 'End_date', 'Year', 'Week', 'Municipality_name',
       'Municipality_code', 'RNA_flow_per_100000_weeklymean',
       'Hospital_admission_notification', 'Hospital_admission', 'Population',
       'popdense', 'Total_reported', 'Deceased',
       'nm_hospital_admission_notification', 'nm_hospital_admission',
       'nm_total_reported', 'nm_deceased', 'Region_code', 'Region_name'], columns='Indicator').reset_index()

In [ ]:
pivot_compliance_df = pivot_compliance_df.rename_axis(None, axis=1)

In [ ]:
pivot_compliance_df

In [ ]:
pivot_compliance_df.to_csv("data/covid19_2020_2022_pivot.csv", encoding='utf-8', index=False)

In [ ]:
new_final_df[(new_final_df['Indicator_category'] == "Naleving")]

In [ ]:
new_final_df[(new_final_df['Indicator_category'] == "Naleving")][['Year', 'Week', 'Municipality_name', 'Municipality_code', 'Indicator_category', 'Indicator', 'Value']]

In [ ]:
new_final_df[new_final_df['popdense'].isna()]['Municipality_name'].unique()

- beemster - 143
- Heerhugowaard - 1528
- Langedijk - 1187
- Uden - 1000
- Landerd - 225
- Mill en Sint Hubert - 211
- Boxmeer - 1200
- Sint Anthonius - 200
- Cuijk - 1900
- Grave - 2600
- Hellevoetsluis - 981
- Brielle - 633
- Westvoorne - 280
- Weesp - 897

In [ ]:
popdense_mun_list = ['Beemster', 'Heerhugowaard', 'Langedijk', 'Uden', 'Landerd', 'Mill en Sint Hubert', 'Boxmeer', 'Sint Anthonis', 'Cuijk', 'Grave', 'Hellevoetsluis', 'Brielle', 'Westvoorne', 'Weesp']

In [ ]:
popdense_data_list = [143, 1528, 1187, 1000, 225, 211, 1200, 200, 1900, 2600, 981, 633, 280, 897]

In [ ]:
for i in range(len(popdense_mun_list)):
    new_final_df.loc[new_final_df['Municipality_name'] == popdense_mun_list[i], 'popdense'] = popdense_data_list[i]

In [ ]:
new_final_df['flag'].unique()

In [ ]:
new_final_df[new_final_df['popdense'].isna()]['Municipality_name'].unique()

In [ ]:
new_final_df.to_csv("data/covid19_2020_2022_nl_new.csv", encoding='utf-8', index=False)

In [ ]:
new_final_df['Indicator'].unique()

In [ ]:
new_final_df[(new_final_df['Indicator_category'] == "Naleving")]['Indicator'].unique()

In [ ]:
mun_df

In [ ]:
mun_code_df

# Create country level data
- Used the output from Shawn's code (add categorical data e.g. curfew, lockdown, variant, mask mandate and pivot table)

## For line graph viz

In [141]:
test_df = pd.read_csv("data/processed_data/data_matrix_110424.csv")

In [142]:
# convert normalized wastewater viral load data to absolute number
test_df['absolute_rna'] = (test_df['RNA_flow_per_100000_weeklymean'] * test_df['Population'])/100000

In [143]:
test_df

,Start_date,End_date,Year,Week,Municipality_name,Municipality_code,RNA_flow_per_100000_weeklymean,Hospital_admission,Population,popdense,...,Zorgen_over_Coronavirus,Hoest_niest_in_elleboog,Thuisgewerkte_uren,indoor_mask_mandate,curfew,runweek,RNAshift,variant,lockstat,absolute_rna
0,2020-10-26,2020-11-01,2020,44,Nunspeet,GM0302,26158709170599,2,27851,223,...,66.2,59.0,66.7,0,0,44,15425096083163,alpha,partial_lockdown,7.285462e+12
1,2020-10-26,2020-11-01,2020,44,Elburg,GM0230,26154476084853,1,23161,377,...,66.2,59.0,66.7,0,0,44,15424141293210,alpha,partial_lockdown,6.057638e+12
2,2020-10-26,2020-11-01,2020,44,Montferland,GM1955,71880313502056,3,36011,349,...,66.2,59.0,66.7,0,0,44,98697168495787,alpha,partial_lockdown,2.588482e+13
3,2020-10-26,2020-11-01,2020,44,Lochem,GM0262,19776353505961,0,33729,161,...,66.2,59.0,66.7,0,0,44,65367999836984,alpha,partial_lockdown,6.670366e+12
4,2020-10-26,2020-11-01,2020,44,Brummen,GM0213,44109851474040,0,20726,252,...,66.2,59.0,66.7,0,0,44,9202252254913,alpha,partial_lockdown,9.142208e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30272,2022-06-27,2022-07-03,2022,26,Hilversum,GM0402,180031919227551,5,90831,2047,...,27.3,69.9,43.6,0,0,131,116722769442392,omicron_ba.5,nolock,-2.094265e+13
30273,2022-06-27,2022-07-03,2022,26,Huizen,GM0406,160772508315075,1,41273,2609,...,27.3,69.9,43.6,0,0,131,67990229021780,omicron_ba.5,nolock,6.635564e+13
30274,2022-06-27,2022-07-03,2022,26,Laren,GM0417,102512022408267,2,11280,944,...,27.3,69.9,43.6,0,0,131,53382356045373,omicron_ba.5,nolock,1.156336e+13
30275,2022-06-27,2022-07-03,2022,26,Wijdemeren,GM1696,173580652953766,1,24358,519,...,27.3,69.9,43.6,0,0,131,90466698285710,omicron_ba.5,nolock,4.228078e+13


In [144]:
test_df[test_df['Total_reported']==-9999]

,Start_date,End_date,Year,Week,Municipality_name,Municipality_code,RNA_flow_per_100000_weeklymean,Hospital_admission,Population,popdense,...,Zorgen_over_Coronavirus,Hoest_niest_in_elleboog,Thuisgewerkte_uren,indoor_mask_mandate,curfew,runweek,RNAshift,variant,lockstat,absolute_rna
189,2020-10-26,2020-11-01,2020,44,Heerhugowaard,GM0398,35313930802917,-9999,57587,1528,...,62.8,59.0,58.5,0,0,44,104795237995403,alpha,partial_lockdown,2.033623e+13
194,2020-10-26,2020-11-01,2020,44,Langedijk,GM0416,35313930802917,-9999,28163,1187,...,62.8,59.0,58.5,0,0,44,104795237995403,alpha,partial_lockdown,9.945462e+12
236,2020-10-26,2020-11-01,2020,44,Uden,GM0856,24569621024743,-9999,42119,1000,...,68.9,65.0,63.8,0,0,44,141007173492936,alpha,partial_lockdown,1.034848e+13
238,2020-10-26,2020-11-01,2020,44,Landerd,GM1685,50411970457542,-9999,15730,225,...,68.9,65.0,63.8,0,0,44,102842265066442,alpha,partial_lockdown,7.929803e+12
241,2020-10-26,2020-11-01,2020,44,Mill en Sint Hubert,GM0815,36420151015096,-9999,10939,211,...,68.9,65.0,63.8,0,0,44,141007173492936,alpha,partial_lockdown,3.984000e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29889,2022-06-20,2022-06-26,2022,25,Brielle,GM0501,139167304784233,-9999,17271,633,...,32.2,63.8,41.4,0,0,130,91142867057691,omicron_ba.5,nolock,2.403559e+13
29891,2022-06-20,2022-06-26,2022,25,Westvoorne,GM0614,204751726409729,-9999,14731,280,...,32.2,63.8,41.4,0,0,130,135250474820284,omicron_ba.5,nolock,3.016198e+13
30224,2022-06-27,2022-07-03,2022,26,Hellevoetsluis,GM0530,182897032403749,-9999,40142,981,...,32.2,63.8,41.4,0,0,131,109461446316439,omicron_ba.5,nolock,7.341853e+13
30229,2022-06-27,2022-07-03,2022,26,Brielle,GM0501,188487495445744,-9999,17271,633,...,32.2,63.8,41.4,0,0,131,139167304784233,omicron_ba.5,nolock,3.255368e+13


In [145]:
# replace -9999 value with 0 so we can aggregate the data
test_df.loc[test_df['Total_reported'] == -9999, 'Total_reported'] = 0
test_df.loc[test_df['Deceased'] == -9999, 'Deceased'] = 0
test_df.loc[test_df['Hospital_admission'] == -9999, 'Hospital_admission'] = 0

In [146]:
# aggregate the data by year and week
agg_reported_case_data = test_df.groupby(['Year', 'Week'])['Total_reported', 'Hospital_admission', 'Deceased', 'absolute_rna'].sum().reset_index()

C:\Users\japan\AppData\Local\Temp\ipykernel_51044\2080188777.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg_reported_case_data = test_df.groupby(['Year', 'Week'])['Total_reported', 'Hospital_admission', 'Deceased', 'absolute_rna'].sum().reset_index()


In [147]:
agg_reported_case_data

,Year,Week,Total_reported,Hospital_admission,Deceased,absolute_rna
0,2020,44,62964,1796,356,5.282654e+15
1,2020,45,47497,1582,525,3.801394e+15
2,2020,46,33183,1314,424,4.602867e+15
3,2020,47,34684,1259,380,2.892963e+15
4,2020,48,31226,1017,428,2.551342e+15
...,...,...,...,...,...,...
83,2022,22,8786,253,7,4.516344e+15
84,2022,23,12718,354,10,6.044624e+15
85,2022,24,22515,472,8,7.344147e+15
86,2022,25,31345,626,9,7.827783e+15


In [148]:
agg_reported_case_data.to_csv("data/processed_data/covid19_country_level_new.csv")

# Country level data for map creation

## For map creation

In [149]:
week_data = pd.read_csv("data/processed_data/data_matrix_110424.csv")

In [150]:
# create a list of year and week that we want
year_list = [2022, 2022, 2021, 2021, 2020, 2020]
week_list = [6, 5, 28, 27, 52, 51]

In [151]:
year_list = [2022, 2022, 2021, 2021, 2020, 2020]
week_list = [19, 18, 52, 51, 46, 45]

In [152]:
# select wastewater viral load and normalized total reported cases from specific week
for i in range(len(week_list)):
    if i>0:
        viz_week = pd.merge(viz_week, week_data[(week_data['Week']==week_list[i])&(week_data['Year']==year_list[i])][['RNA_flow_per_100000_weeklymean', 'nm_total_reported', 'Municipality_code']], how="left", on=['Municipality_code'])
        viz_week = viz_week.rename({'RNA_flow_per_100000_weeklymean': 'rna_'+ str(year_list[i]) +'_'+str(week_list[i]), 'nm_total_reported': 'nm_report_case_'+str(year_list[i])+'_'+str(week_list[i])}, axis=1)
    else:
        viz_week = pd.merge(mun_code_df, week_data[(week_data['Week']==week_list[i])&(week_data['Year']==year_list[i])][['RNA_flow_per_100000_weeklymean', 'nm_total_reported', 'Municipality_code']], how="left", on=['Municipality_code'])
        viz_week = viz_week.rename({'RNA_flow_per_100000_weeklymean': 'rna_'+ str(year_list[i]) +'_'+str(week_list[i]), 'nm_total_reported': 'nm_report_case_'+str(year_list[i])+'_'+str(week_list[i])}, axis=1)

In [153]:
viz_week

,Municipality_name,Municipality_code,Population,rna_2022_19,nm_report_case_2022_19,rna_2022_18,nm_report_case_2022_18,rna_2021_52,nm_report_case_2021_52,rna_2021_51,nm_report_case_2021_51,rna_2020_46,nm_report_case_2020_46,rna_2020_45,nm_report_case_2020_45
0,Nunspeet,GM0302,27851,2.978526e+13,46.676960,2.282095e+13,28.724283,6.732613e+13,398.549424,33218460093445,455.997989,9.425909e+12,154.393020,3.827245e+13,179.526767
1,Valkenswaard,GM0858,31193,1.295092e+13,19.235085,1.548888e+13,41.676017,7.198732e+13,484.082967,58470342308111,346.231526,6.187165e+13,185.939153,5.611317e+13,243.644407
2,Wierden,GM0189,24446,2.890815e+13,24.543893,3.775824e+13,44.997137,8.390587e+13,666.775751,64366297627942,359.977092,8.631840e+13,433.608770,8.074795e+13,556.328234
3,Hengelo (O),GM0164,81140,1.510126e+13,41.902884,2.462429e+13,57.924575,4.342535e+13,496.672418,41376426178736,407.936899,2.693717e+13,268.671432,8.298401e+13,298.249938
4,Renkum,GM0274,31419,2.043570e+13,50.924600,2.681317e+13,70.021325,4.103444e+13,372.386136,43538614649555,461.504185,5.912121e+13,130.494287,6.731668e+13,213.246762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,Gennep,GM0907,16921,1.513901e+13,5.909816,2.522403e+13,11.819632,4.908922e+13,413.687134,52679612097107,319.130075,1.785029e+13,147.745405,5.715011e+13,372.318421
348,Eijsden-Margraten,GM1903,25768,2.954298e+13,23.284694,2.920038e+13,7.761565,6.347214e+13,558.832661,50069452802512,469.574666,1.982198e+13,124.185036,1.420363e+13,213.443030
349,Tholen,GM0716,25757,7.657495e+12,15.529759,1.244912e+13,27.177078,4.941833e+13,574.601079,71243956116642,543.541562,NaN,NaN,-9.999000e+03,314.477618
350,Dalfsen,GM0148,28587,2.722180e+13,34.980935,1.654528e+13,59.467590,5.202748e+13,580.683527,45651464694821,458.250254,2.507627e+13,223.877986,2.289193e+13,230.874174


In [154]:
viz_week.to_csv("data/processed_data/viz_covid19_week_new_v2.1.csv", encoding='utf-8', index=False)